In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
# Read the data using Dask
df_user = pd.read_csv('/home/nkama/masters_thesis_project/thesis/data/event_rec_engine_challenge/users.csv')
                    

In [ ]:
df_user.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0


In [4]:
"""
Load the saved user_event_pairs dataframe so we don't have to rerun all the code needed
to create the dataframe should the kernel crash when processing large data volume.
"""

user_event_pairs = pd.read_csv("interactions_data.csv")

user_event_pairs.head()

,Unnamed: 0,event_id,user_id,interaction_type,interaction_label
0,0,1159822043,1975964455,yes,1
1,1,1159822043,252302513,yes,1
2,2,1159822043,4226086795,yes,1
3,3,1159822043,3805886383,yes,1
4,4,1159822043,1420484491,yes,1


In [10]:
unique_users = user_event_pairs['user_id'].unique()

In [11]:
# Convert both to string type to ensure matching
user_event_pairs['user_id'] = user_event_pairs['user_id'].astype(str)
df_user['user_id'] = df_user['user_id'].astype(str)


In [12]:
# Filter users data
filtered_user_data = df_user[df_user["user_id"].isin(unique_users)]
filtered_user_data.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0
7,3473687777,id_ID,1965,female,2012-10-03T12:19:29.975Z,Medan Indonesia,420.0
12,1355996271,id_ID,1993,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,420.0


In [13]:
filtered_user_data.isnull().sum()

user_id         0
locale          0
birthyear     234
gender         42
joinedAt       17
location     1429
timezone       38
dtype: int64

In [14]:
filtered_user_data = filtered_user_data.drop(["timezone","locale"],axis=1)

In [15]:
filtered_user_data = filtered_user_data.dropna()
filtered_user_data

,user_id,birthyear,gender,joinedAt,location
0,3197468391,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia
1,3537982273,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia
7,3473687777,1965,female,2012-10-03T12:19:29.975Z,Medan Indonesia
12,1355996271,1993,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia
13,2411726276,1996,female,2012-10-30T05:16:27.136Z,Triolet Mauritius
...,...,...,...,...,...
38187,3566369790,1989,male,2012-10-30T08:56:19.611Z,Jogjakarta Indonesia
38190,2630548270,1988,male,2012-10-29T16:02:19.692Z,Yogyakarta
38199,280076614,1995,female,2012-11-22T12:06:08.668Z,Phnom Penh
38203,3890944219,1993,female,2012-11-04T07:18:39.354Z,Kebumen Jawa Tengah Indonesia


In [16]:
len(filtered_user_data)

9922

In [18]:
# Print summary information
print(f"Original users: {len(df_user)}, Filtered users: {len(filtered_user_data)}")

Original users: 38209, Filtered users: 9922


Extract age column from the birtday and event start time

In [19]:
events_data = pd.read_csv("/home/nkama/masters_thesis_project/thesis/data/event_rec_engine_challenge/events.csv",\
                           usecols=range(3))
#events_data['start_time'] = pd.to_datetime(events_data['start_time'], errors='coerce')
events_data = events_data.dropna(subset=['start_time'])

In [20]:
from datetime import datetime


# Create a copy of the filtered data to avoid the warning
filtered_user_data = filtered_user_data.copy()

# Convert birthyear to int and calculate age
filtered_user_data.loc[:, 'birthyear'] = filtered_user_data['birthyear'].astype(int)
# First, convert string timestamps to datetime objects
events_data['start_time'] = pd.to_datetime(events_data['start_time'],errors='coerce')
#filtered_user_data['joinedAt'] = pd.to_datetime(filtered_user_data['joinedAt'])

# Extract reference year - let's use the median event year as our reference point
# This is a balanced approach that represents the "typical" time in the dataset
median_event_date = events_data['start_time'].median()
reference_year = median_event_date.year
filtered_user_data.loc[:, 'age'] = reference_year - filtered_user_data['birthyear']

# Verify the changes
print("Sample of data with age:")
print(filtered_user_data[['user_id', 'birthyear', 'age']].head())

# Check for any invalid ages
invalid_ages = filtered_user_data[(filtered_user_data['age'] < 0) | (filtered_user_data['age'] > 100)]
print(f"\nNumber of invalid ages: {len(invalid_ages)}")



Sample of data with age:
       user_id birthyear age
0   3197468391      1993  19
1   3537982273      1992  20
7   3473687777      1965  47
12  1355996271      1993  19
13  2411726276      1996  16

Number of invalid ages: 30


In [21]:
len(filtered_user_data)

9922

In [22]:
filtered_user_data.isnull().sum()

user_id      0
birthyear    0
gender       0
joinedAt     0
location     0
age          0
dtype: int64

In [23]:
filtered_user_data = filtered_user_data.drop("birthyear",axis=1)


In [24]:
filtered_user_data.head()


,user_id,gender,joinedAt,location,age
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20
7,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47
12,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19
13,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16


In [25]:
filtered_user_data.reset_index(drop=True)

,user_id,gender,joinedAt,location,age
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20
2,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47
3,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19
4,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16
...,...,...,...,...,...
9917,3566369790,male,2012-10-30T08:56:19.611Z,Jogjakarta Indonesia,23
9918,2630548270,male,2012-10-29T16:02:19.692Z,Yogyakarta,24
9919,280076614,female,2012-11-22T12:06:08.668Z,Phnom Penh,17
9920,3890944219,female,2012-11-04T07:18:39.354Z,Kebumen Jawa Tengah Indonesia,19


In [113]:
filtered_user_data.to_csv("filtered_user_data.csv")

In [26]:
from get_user_location_data import add_user_coordinates
import pandas as pd


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import time


df = filtered_user_data.copy()
# Initialize geocoder
geolocator = Nominatim(user_agent="location_coordinates_finder")

# Function to get latitude and longitude
def get_coordinates(location):
    try:
        # Add small delay to respect geocoding service rate limits
        time.sleep(1)
        location_data = geolocator.geocode(location)
        if location_data:
            return location_data.latitude, location_data.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
        return None, None

# Add latitude and longitude columns
df[['lat', 'lon']] = df['location'].apply(lambda x: pd.Series(get_coordinates(x)))

# Display the result
#df

In [102]:
df.head()

,user_id,gender,joinedAt,location,age
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20
7,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47
12,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19
13,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16


In [106]:
# import the add_user_coordinates function to retrieve user cordinates
from get_user_location_data import add_user_coordinates

In [ ]:
users_df = add_user_coordinates(filtered_user_data[["location"]])

In [20]:
filtered_user_data = pd.read_csv("/home/nkama/masters_thesis_project/thesis/filtered_user_data.csv")
users_location_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/users_location.csv")

In [29]:
len(filtered_user_data), len(users_location_df)

(9922, 9922)

In [23]:
users_location_df.head()

,Unnamed: 0,location,lat,lng
0,0,Medan Indonesia,3.590,98.674
1,1,Medan Indonesia,3.590,98.674
2,7,Medan Indonesia,3.590,98.674
3,12,Djokja Yogyakarta Indonesia,NaN,NaN
4,13,Triolet Mauritius,-20.057,57.552


In [24]:
users_data = filtered_user_data.merge(users_df).drop("Unnamed: 0",axis=1)

In [25]:
users_data.head()

,user_id,gender,joinedAt,location,age,lat,lng
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.590,98.674
1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20,3.590,98.674
2,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47,3.590,98.674
3,1355996271,female,2012-10-26T15:34:46.113Z,Djokja Yogyakarta Indonesia,19,NaN,NaN
4,2411726276,female,2012-10-30T05:16:27.136Z,Triolet Mauritius,16,-20.057,57.552


In [18]:
len(users_data)

9922

In [26]:
users_data.isnull().sum()

user_id       0
gender        0
joinedAt      0
location      0
age           0
lat         398
lng         398
dtype: int64

In [27]:
users_data = users_data.dropna()

In [28]:
len(users_data)

9524

In [22]:
#users_data.to_csv("users_data.csv")

## Create user onboarding data

In [67]:
import pandas as pd
users_df = pd.read_csv("users_data.csv")
events_df = pd.read_csv("complete_events_data.csv")
interactions_df = pd.read_csv("interactions_data.csv")

In [17]:
users_df["age"].min()

np.int64(13)

In [3]:
len(events_df)

11627

In [4]:
events_df.category.nunique()

12

In [68]:
events_df.head(2)

,Unnamed: 0,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_description,category,title,event_type
0,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor
1,1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,162,112,1021,150,1445,Cloudy,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor


In [30]:
events_df["city"].nunique()

1894

In [69]:
#col = ["user_id", "gender"]
#users_df = users_df[col]
#col2 = ["event_id","city","category"]
#events_df = events_df[col2]

# Corrected merge to include all three DataFrames
interaction_users_events = interactions_df[['event_id','user_id']].merge(users_df, on='user_id', how='inner') \
                                           .merge(events_df, on='event_id', how='inner')

len(interaction_users_events)




28249

In [8]:
interaction_users_events['event_id'].nunique()

4967

In [33]:
interaction_users_events['user_id'].nunique()

5315

In [71]:
#interaction_users_events = interaction_users_events.drop("Unnamed: 0",axis=1)
interaction_users_events.head(3)

,event_id,user_id,Unnamed: 0_x,gender,joinedAt,location,age,lat_x,lng_x,Unnamed: 0_y,...,lng_y,yes_count,maybe_count,invited_count,no_count,total_users,weather_description,category,title,event_type
0,702719295,3468617687,749,female,2012-09-25T17:48:37.804Z,Saint Catharines Ontario,20,43.158,-79.244,5875,...,-73.561,121,28,93,22,264,Cloudy,Food & Drink,Montreal Spring Bites Night Market | Food Truc...,Indoor
1,702719295,487910947,1748,male,2012-09-25T16:28:31.617Z,Toronto Ontario,22,43.653,-79.384,5875,...,-73.561,121,28,93,22,264,Cloudy,Food & Drink,Montreal Spring Bites Night Market | Food Truc...,Indoor
2,702719295,2769572227,2150,female,2012-09-25T15:40:36.962Z,Toronto Ontario,24,43.653,-79.384,5875,...,-73.561,121,28,93,22,264,Cloudy,Food & Drink,Montreal Spring Bites Night Market | Food Truc...,Indoor


In [72]:
interaction_users_events[['event_id', 'user_id', 'gender', 'location',
 'lat_x', 'lng_x', 'city', 'lat_y',
       'lng_y', 'joinedAt']].head(10)

,event_id,user_id,gender,location,lat_x,lng_x,city,lat_y,lng_y,joinedAt
0,702719295,3468617687,female,Saint Catharines Ontario,43.158,-79.244,Montreal,45.505,-73.561,2012-09-25T17:48:37.804Z
1,702719295,487910947,male,Toronto Ontario,43.653,-79.384,Montreal,45.505,-73.561,2012-09-25T16:28:31.617Z
2,702719295,2769572227,female,Toronto Ontario,43.653,-79.384,Montreal,45.505,-73.561,2012-09-25T15:40:36.962Z
3,702719295,2974942354,male,Toronto Ontario,43.653,-79.384,Montreal,45.505,-73.561,2012-10-11T18:00:17.768Z
4,3093414446,4137863751,female,San Francisco California,37.779,-122.419,San Francisco,37.784,-122.409,2012-10-08T22:00:48.878Z
5,1811694804,2192466031,male,Vado Ligure,44.271,8.438,Genova,44.403,8.946,2012-11-19T15:56:54.372Z
6,1868124810,3226978978,female,Toronto Ontario,43.653,-79.384,Toronto,43.661,-79.382,2012-09-25T14:22:14.718Z
7,1868124810,2716313754,male,Toronto Ontario,43.653,-79.384,Toronto,43.661,-79.382,2012-09-23T18:47:10.426Z
8,2971099513,2693391182,female,Calgary AB,51.046,-114.058,Calgary,51.043,-114.075,2012-10-05T01:11:13.442Z
9,804619862,1393131093,male,Los Angeles California,34.054,-118.243,Los Angeles,34.020,-118.286,2012-11-19T11:39:22.102Z


In [51]:
interaction_users_events["joinedAt"].unique()

array(['2012-09-25T17:48:37.804Z', '2012-09-25T16:28:31.617Z',
       '2012-09-25T15:40:36.962Z', ..., '2012-10-02T15:23:18.359Z',
       '2012-10-26T09:36:33.227Z', '2012-10-28T05:27:42.970Z'],
      shape=(5315,), dtype=object)

In [109]:
# Count the number of unique users for each event
attendance_count = interaction_users_events.groupby('event_id')['user_id'].nunique().reset_index()

# Rename the columns for clarity
attendance_count.rename(columns={'user_id': 'user_count'}, inplace=True)

# Display the result
print(attendance_count)

        event_id  user_count
0         104070           3
1         319291           7
2         462005           8
3         847179           1
4        1204184           1
...          ...         ...
4962  4290470709           2
4963  4293147981           9
4964  4293596113          10
4965  4294096869           7
4966  4294936792           1

[4967 rows x 2 columns]


In [116]:
len(attendance_count[attendance_count["user_count"]==5])

260

In [73]:
event_col = events_df.columns
event_col

Index(['Unnamed: 0', 'event_id', 'start_time', 'city', 'lat', 'lng',
       'yes_count', 'maybe_count', 'invited_count', 'no_count', 'total_users',
       'weather_description', 'category', 'title', 'event_type'],
      dtype='object')

In [74]:
col = ['event_id', 'start_time', 'city', 'lat', 'lng','weather_description', 'category', 'title', 'event_type']
events_df = events_df[col]

interactions = interactions_df[["event_id","user_id","interaction_type"]].merge(events_df, on='event_id', how='inner')
                                


In [75]:
interactions.head(2)

,event_id,user_id,interaction_type,start_time,city,lat,lng,weather_description,category,title,event_type
0,1186208412,3320380166,maybe,2012-08-08T12:00:00.000Z,Toronto,43.784,-79.187,Cloudy,Education & Learning,Learning Under the Sun Toronto Summer Academy,Indoor
1,1186208412,3810793697,maybe,2012-08-08T12:00:00.000Z,Toronto,43.784,-79.187,Cloudy,Education & Learning,Learning Under the Sun Toronto Summer Academy,Indoor


In [3]:
synth = pd.read_csv("/home/nkama/masters_thesis_project/thesis/processed_events_checkpoint.csv")
synth.head()

,Unnamed: 0,event_id,user_id,interaction_type,start_time,city,lat,lng,weather_description,category,title,event_type,gender,joinedAt,location,age,lat_user,lng_user,user_distance_km_from_event,interaction_id
0,2888814,3540042059,27989010094,yes,2012-12-15 00:00:00.001000+00:00,Kalasan,-7.753,110.493,Heavy Rain,Seasonal & Festivals,Summer Nights in Kalasan - A Festival Under th...,Indoor,male,2012-10-02 12:51:40+00:00,Mandaue City,60,10.286715,124.090473,2507.525560,3540042059_27989010094
1,1735086,3619841782,4109556373,maybe,2012-12-13 21:00:00.002000+00:00,Pawtucket,41.874,-71.365,Cloudy,Arts & Culture,Winter Nights in Pawtucket - A Celebration of ...,Indoor,female,2012-09-26 06:18:41+00:00,Paia,37,21.089553,-156.595789,9737.857738,3619841782_4109556373
2,1626704,3312755383,28004789702,invited & yes,2012-10-03 15:00:00.003000+00:00,Jakarta,-6.289,106.778,Light Rain,Education & Learning,Spring Learning Jam Jakarta 2025 | Ignite Your...,Indoor,male,2012-09-10 02:59:10+00:00,Wiesbaden,36,50.102090,8.151449,12610.670721,3312755383_28004789702
3,2051286,3439956212,54915887936,invited & yes,2012-10-23 21:30:00.003000+00:00,Sunny Isles,25.931,-80.121,Cloudy,Seasonal & Festivals,Autumn Nights in Sunny Isles - Harvest Moon Fe...,Indoor,female,2012-09-01 19:34:56+00:00,Methuen,73,42.803209,-71.036605,2127.026710,3439956212_54915887936
4,4798331,1064673805,38141420251,maybe,2012-12-15 00:00:00.003000+00:00,Ulu Tiram,1.605,103.822,Rain,Seasonal & Festivals,Winter Magic Nights in Ulu Tiram,Indoor,male,2012-10-18 01:34:19+00:00,Semenyih,36,2.760850,102.007776,238.776419,1064673805_38141420251


In [4]:
len(synth)

100000

In [14]:
synth["category"].unique()

array(['Seasonal & Festivals', 'Arts & Culture', 'Education & Learning',
       'Sports & Fitness', 'Health & Wellness', 'Technology',
       'Business & Networking', 'Entertainment', 'Community & Causes',
       'Food & Drink', 'Immersive Experiences', 'Music & Concerts'],
      dtype=object)

In [15]:
import pandas as pd

# Define a function to determine user weather preference
def determine_weather_preference(group):
    event_types = group['event_type'].unique()
    if 'Indoor' in event_types and 'Outdoor' in event_types:
        return 'any'
    elif 'Indoor' in event_types:
        return 'indoor'
    elif 'Outdoor' in event_types:
        return 'outdoor'
    else:
        return 'any'  # Default case if no events are found
df = synth.copy()
# Group by user_id and apply the function
user_preferences = df.groupby('user_id').apply(determine_weather_preference).reset_index()

# Rename the columns for clarity
user_preferences.columns = ['user_id', 'user_weather_preference']

# Merge the preferences back into the original DataFrame
df = df.merge(user_preferences, on='user_id', how='left')

# Display the updated DataFrame
print(df[['user_id', 'user_weather_preference']].drop_duplicates())

           user_id user_weather_preference
0      27989010094                  indoor
1       4109556373                  indoor
2      28004789702                  indoor
3      54915887936                  indoor
4      38141420251                  indoor
...            ...                     ...
99980  26777977446                  indoor
99988  26501010491                  indoor
99989  78208885784                  indoor
99990  21282415810                  indoor
99998  25141729863                  indoor

[39926 rows x 2 columns]


/tmp/ipykernel_345514/747041535.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_preferences = df.groupby('user_id').apply(determine_weather_preference).reset_index()


In [16]:
df.head()

,Unnamed: 0,event_id,user_id,interaction_type,start_time,city,lat,lng,weather_description,category,...,event_type,gender,joinedAt,location,age,lat_user,lng_user,user_distance_km_from_event,interaction_id,user_weather_preference
0,2888814,3540042059,27989010094,yes,2012-12-15 00:00:00.001000+00:00,Kalasan,-7.753,110.493,Heavy Rain,Seasonal & Festivals,...,Indoor,male,2012-10-02 12:51:40+00:00,Mandaue City,60,10.286715,124.090473,2507.525560,3540042059_27989010094,indoor
1,1735086,3619841782,4109556373,maybe,2012-12-13 21:00:00.002000+00:00,Pawtucket,41.874,-71.365,Cloudy,Arts & Culture,...,Indoor,female,2012-09-26 06:18:41+00:00,Paia,37,21.089553,-156.595789,9737.857738,3619841782_4109556373,indoor
2,1626704,3312755383,28004789702,invited & yes,2012-10-03 15:00:00.003000+00:00,Jakarta,-6.289,106.778,Light Rain,Education & Learning,...,Indoor,male,2012-09-10 02:59:10+00:00,Wiesbaden,36,50.102090,8.151449,12610.670721,3312755383_28004789702,indoor
3,2051286,3439956212,54915887936,invited & yes,2012-10-23 21:30:00.003000+00:00,Sunny Isles,25.931,-80.121,Cloudy,Seasonal & Festivals,...,Indoor,female,2012-09-01 19:34:56+00:00,Methuen,73,42.803209,-71.036605,2127.026710,3439956212_54915887936,indoor
4,4798331,1064673805,38141420251,maybe,2012-12-15 00:00:00.003000+00:00,Ulu Tiram,1.605,103.822,Rain,Seasonal & Festivals,...,Indoor,male,2012-10-18 01:34:19+00:00,Semenyih,36,2.760850,102.007776,238.776419,1064673805_38141420251,indoor


In [76]:
len(interactions)

6983851

In [77]:
interactions.isnull().sum()

event_id               0
user_id                0
interaction_type       0
start_time             0
city                   0
lat                    0
lng                    0
weather_description    0
category               0
title                  0
event_type             0
dtype: int64

In [78]:
len(interactions.event_id.unique()), len(interactions.user_id.unique())

(11627, 2019913)

In [91]:
unique_users =interaction_users_events['user_id'].unique().tolist()
unique_events =interaction_users_events['event_id'].unique().tolist()

In [16]:
len(unique_users)

5315

In [108]:
len(unique_events)

4967

In [118]:
users_2_gen = interactions[~interactions["event_id"].isin(unique_events)]
users_2_gen.head(2)

,event_id,user_id,interaction_type,start_time,city,lat,lng,weather_description,category,title,event_type
0,1186208412,3320380166,maybe,2012-08-08T12:00:00.000Z,Toronto,43.784,-79.187,Cloudy,Education & Learning,Learning Under the Sun Toronto Summer Academy,Indoor
1,1186208412,3810793697,maybe,2012-08-08T12:00:00.000Z,Toronto,43.784,-79.187,Cloudy,Education & Learning,Learning Under the Sun Toronto Summer Academy,Indoor


In [119]:
len(users_2_gen)

1191361

In [120]:
ids_2_gen = users_2_gen["user_id"].unique().tolist()#[:5000]
event_ids_2_gen = users_2_gen["event_id"].unique().tolist()#[:5000]

In [121]:
len(ids_2_gen), len(event_ids_2_gen)

(833507, 6660)

In [81]:
users_2_gen['event_id'].nunique(), users_2_gen['user_id'].nunique()

(11626, 2014598)

In [45]:
users_2_gen["city"].unique()

array(['Toronto', 'Kamloops', 'Chicago', ..., 'Norco', 'Cayuga',
       'San Ramon'], shape=(1894,), dtype=object)

In [124]:
users_2_gen.to_csv("users_2_gen.csv")

In [28]:
users_2_gen_2 = events_df[events_df["event_id"].isin(event_ids_2_gen)]
users_2_gen_2

,event_id,start_time,city,lat,lng,weather_description,category,title,event_type
0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor
1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,Cloudy,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor
2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,Cloudy,Entertainment,Karachi Nights: Summer Fiesta Under the Stars,Indoor
3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,Light Rain,Arts & Culture,Funky Art Walk LA - Morning Vibes & Visual Del...,Indoor
4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,Cloudy,Arts & Culture,Palo Alto Art After Dark - A Fall Fiesta of Cr...,Indoor
...,...,...,...,...,...,...,...,...,...
11622,3267865388,2012-09-15T23:00:00.003Z,Tempe,33.336,-111.938,Sunny,Music & Concerts,Tempe Night Vibes - Fall Music Fest ft. Local ...,Outdoor
11623,3409015015,2012-09-28T21:00:00.003Z,Kitchener,43.428,-80.434,Cloudy,Entertainment,"Fall Night Fiesta in Kitchener - Beats, Bites,...",Indoor
11624,3119357029,2012-10-31T00:00:00.001Z,London,51.481,-0.191,Drizzle,Arts & Culture,Autumn Arts Awakening London Nights 2025,Indoor
11625,3323611071,2012-12-15T02:00:00.003Z,Los Angeles,34.034,-118.206,Drizzle,Arts & Culture,Luminous Nights: Winter Art Showcase in LA,Indoor


In [24]:
users_2_gen_2['event_id'].nunique()

4

In [54]:
len(users_2_gen)

162145

In [55]:
users_2_gen["user_id"].nunique()

5000

In [1]:
events_df.head()

NameError: name 'events_df' is not defined

In [75]:
users_2_gen = users_2_gen.drop("user_id",axis=1)
users_2_gen.columns

Index(['event_id', 'interaction_type', 'start_time', 'city', 'lat', 'lng',
       'weather_description', 'category', 'title', 'event_type'],
      dtype='object')

I have a dataframe of events (11627 rows) with the followin columns:
['event_id', 'interaction_type', 'start_time', 'city', 'lat', 'lng',
       'weather_description', 'category', 'title', 'event_type'].
For each event in each row, I would like to synthesize user data with the following column:
['user_id',	'gender',	'joinedAt',	'location',	'age',	'lat',	'lng']
The average is 22 and gender proportion:
male      5203
female    4321
The events data would then have an additional column called "user_id" corresponding to the event the 
user attended.
Note: total number of users to synthesize = 5000.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from sklearn.cluster import KMeans

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

EVENT_CATEGORIES = [
    'Music & Concerts',
    'Food & Drink',
    'Education & Learning',
    'Sports & Fitness',
    'Arts & Culture',
    'Business & Networking',
    'Technology',
    'Community & Causes',
    'Health & Wellness',
    'Entertainment',
    'Seasonal & Festivals',
    'Immersive Experiences'
]

def extract_locations_from_events(events_df, num_clusters=300):
    """
    Extract location information from events data and cluster
    into representative locations for user generation

    Parameters:
    -----------
    events_df : DataFrame
        Events dataframe with lat/lng coordinates
    num_clusters : int
        Number of location clusters to create

    Returns:
    --------
    List of tuples (location_name, lat, lng) to use for user generation
    """
    # Filter events with valid coordinates
    valid_events = events_df.dropna(subset=['lat', 'lng'])

    # If city column exists, use it
    if 'city' in valid_events.columns:
        # Group by city and get average coordinates
        city_groups = valid_events.groupby('city').agg({
            'lat': 'mean',
            'lng': 'mean'
        }).reset_index()

        # Convert to list of tuples
        locations = [(city, lat, lng) for city, lat, lng in
                     zip(city_groups['city'], city_groups['lat'], city_groups['lng'])]
    else:
        # Use KMeans clustering to identify geographic clusters
        coords = valid_events[['lat', 'lng']].values

        # Adjust num_clusters if we have fewer data points than requested clusters
        actual_clusters = min(num_clusters, len(coords))

        if actual_clusters <= 1:
            # Handle edge case with very little data
            return [("Default Location", coords[0][0], coords[0][1])]

        # Apply KMeans clustering
        kmeans = KMeans(n_clusters=actual_clusters, random_state=42, n_init=10)  # Added n_init
        kmeans.fit(coords)

        # Get cluster centers
        centers = kmeans.cluster_centers_

        # Create location names based on cluster ID
        locations = [(f"Cluster_{i}", center[0], center[1])
                     for i, center in enumerate(centers)]

    print(f"Extracted {len(locations)} distinct locations from event data")
    return locations

def generate_synthetic_users(n=5000, locations=None, gender_distribution={'male': 0.55, 'female': 0.45},
                          mean_age=22, age_std=5):
    """
    Generate synthetic user data based on event locations

    Parameters:
    -----------
    n : int
        Number of users to generate
    locations : list of tuples
        List of (location_name, lat, lng) tuples to use
    gender_distribution : dict
        Distribution of genders
    mean_age : int
        Mean age of users
    age_std : int
        Standard deviation of age

    Returns:
    --------
    DataFrame with synthetic user data
    """
    # If no locations provided, use a default set
    if locations is None or len(locations) == 0:
        locations = [
            ("Jakarta", -6.200, 106.816),
            ("Singapore", 1.352, 103.820),
            ("New York", 40.712, -74.006),
            ("London", 51.507, -0.127)
        ]

    # Initialize lists for each column
    user_ids = []
    genders = []
    joined_ats = []
    user_locations = []
    ages = []
    lats = []
    lngs = []

    # Generate join dates between 2012-09-01 and 2012-10-31
    start_date = datetime(2012, 9, 1)
    end_date = datetime(2012, 10, 31)

    # Generate genders based on distribution
    gender_choices = np.random.choice(
        list(gender_distribution.keys()),
        size=n,
        p=list(gender_distribution.values())
    )

    # Weight locations based on their frequency (proxy for population)
    # You could further refine this with actual population data if available
    location_weights = np.ones(len(locations)) / len(locations)

    for i in range(n):
        # Generate a random numeric ID between 8 and 11 digits
        user_id = str(random.randint(10000000, 99999999999))

        # Get gender from pre-generated choices
        gender = gender_choices[i]

        # Generate random join date
        join_date = start_date + timedelta(
            seconds=random.randint(0, int((end_date - start_date).total_seconds()))
        )
        joined_at = join_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"

        # Select random location based on weights
        location_idx = np.random.choice(range(len(locations)), p=location_weights)
        location_name, base_lat, base_lng = locations[location_idx]

        # Add small random variations to coordinates (people in same city aren't at exact same location)
        # Scale the variation based on city size (rough proxy)
        variation_scale = 0.02  # About 2km at equator
        lat = base_lat + random.uniform(-variation_scale, variation_scale)
        lng = base_lng + random.uniform(-variation_scale, variation_scale)

        # Occasionally set coordinates to NaN (about 5% of the time)
        if random.random() < 0.05:
            lat, lng = np.nan, np.nan

        # Generate age with a normal distribution around mean_age
        age = max(16, int(np.random.normal(mean_age, age_std))) #Ensuring age is not less than 16

        # Append to lists
        user_ids.append(user_id)
        genders.append(gender)
        joined_ats.append(joined_at)
        user_locations.append(location_name)
        ages.append(age)
        lats.append(lat)
        lngs.append(lng)

    # Create dataframe
    users_df = pd.DataFrame({
        'user_id': user_ids,
        'gender': genders,
        'joinedAt': joined_ats,
        'location': user_locations,
        'age': ages,
        'lat': lats,
        'lng': lngs
    })

    return users_df

def assign_users_to_events(events_df, users_df, bias_local=True, bias_category=True):
    """
    Assign users to events with realistic biases

    Parameters:
    -----------
    events_df : DataFrame
        Events dataframe
    users_df : DataFrame
        Users dataframe
    bias_local : bool
        Whether to bias users toward nearby events
    bias_category : bool
        Whether to bias users toward certain categories based on demographics

    Returns:
    --------
    Updated events dataframe with user information
    """
    # Create copy of events dataframe
    events_with_users = events_df.copy()

    # Ensure lat/lng are numeric in both dataframes
    for df in [events_with_users, users_df]:
        for col in ['lat', 'lng']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

    # Convert 'start_time' and 'joinedAt' to datetime objects
    if 'start_time' in events_with_users.columns:
        events_with_users['start_time'] = pd.to_datetime(events_with_users['start_time'], errors='coerce')
    users_df['joinedAt'] = pd.to_datetime(users_df['joinedAt'], errors='coerce')

    # Define category biases
    category_biases = {
        'Music & Concerts': {'age_range': (16, 35), 'gender_bias': None},
        'Sports & Fitness': {'age_range': (16, 45), 'gender_bias': 'male'},
        'Arts & Culture': {'age_range': (25, 55), 'gender_bias': 'female'},
        'Technology': {'age_range': (16, 40), 'gender_bias': None},
        'Food & Drink': {'age_range': (16, 60), 'gender_bias': None},
        'Business & Networking': {'age_range': (30, 60), 'gender_bias': None},
        'Education & Learning': {'age_range': (16, 50), 'gender_bias': None},
        'Community & Causes': {'age_range': (16, 60), 'gender_bias': None},
        'Health & Wellness':  {'age_range': (16, 60), 'gender_bias': 'female'},
        'Entertainment': {'age_range': (16, 60), 'gender_bias': None},
        'Seasonal & Festivals': {'age_range': (16, 60), 'gender_bias': None},
        'Immersive Experiences': {'age_range': (16, 60), 'gender_bias': None},
    }

    # Add user information to events
    user_cols = ['user_id', 'gender', 'joinedAt', 'location', 'age', 'lat_user', 'lng_user']
    for col in user_cols:
        events_with_users[col] = None  # Initialize columns

    events_with_users['user_distance_km_from_event'] = None
    events_with_users['interaction_id'] = None

    for idx, event in events_with_users.iterrows():
        if idx % 1000 == 0:
            print(f"Processing event {idx} of {len(events_with_users)}")

        # Filter eligible users based on join date
        if 'start_time' in event and not pd.isna(event['start_time']):
            event_date = event['start_time'].date()
            eligible_users = users_df[users_df['joinedAt'].dt.date < event_date]
        else:
            eligible_users = users_df

        if len(eligible_users) == 0:
            eligible_users = users_df

        # Apply category bias
        if bias_category and 'category' in event and not pd.isna(event['category']):
            category = event['category']  # Use the actual category names
            if category in category_biases:
                bias = category_biases[category]
                if bias['age_range']:
                    min_age, max_age = bias['age_range']
                    eligible_users = eligible_users[(eligible_users['age'] >= min_age) & (eligible_users['age'] <= max_age)]
                if bias['gender_bias']:
                    eligible_users = eligible_users[eligible_users['gender'] == bias['gender_bias']]

        # Apply geographic bias
        if bias_local and 'lat' in event and 'lng' in event and not pd.isna(event['lat']) and not pd.isna(event['lng']):
            valid_users = eligible_users.dropna(subset=['lat', 'lng']).copy()
            if not valid_users.empty:
                valid_users['distance'] = valid_users.apply(
                    lambda user: np.sqrt((user['lat'] - event['lat'])**2 + (user['lng'] - event['lng'])**2),
                    axis=1
                )
                weights = 1 / (valid_users['distance'] + 0.1)
                weights = weights / weights.sum()
                selected_user = valid_users.sample(n=1, weights=weights).iloc[0]
                distance_km = selected_user['distance'] * 111.0
            else:
                selected_user = eligible_users.sample(n=1).iloc[0]
                distance_km = None
        else:
            selected_user = eligible_users.sample(n=1).iloc[0]
            distance_km = None

        # Assign user data to event
        events_with_users.at[idx, 'user_id'] = selected_user['user_id']
        events_with_users.at[idx, 'gender'] = selected_user['gender']
        events_with_users.at[idx, 'joinedAt'] = selected_user['joinedAt']
        events_with_users.at[idx, 'location'] = selected_user['location']
        events_with_users.at[idx, 'age'] = selected_user['age']
        events_with_users.at[idx, 'lat_user'] = selected_user['lat']
        events_with_users.at[idx, 'lng_user'] = selected_user['lng']
        events_with_users.at[idx, 'user_distance_km_from_event'] = distance_km

        # Create interaction_id
        events_with_users.at[idx, 'interaction_id'] = f"{event['event_id']}_{selected_user['user_id']}"

    return events_with_users

def main():
    # Load original events data
    try:
        events_df = pd.read_csv("complete_events_data.csv")
        events_df = events_df.iloc[:10]
        print(f"Loaded {len(events_df)} events")
    except FileNotFoundError:
        print("Events data file not found. Creating sample data for demonstration.")
        n_events = 1000
        events_df = pd.DataFrame({
            'event_id': range(1, n_events + 1),
            'interaction_type': np.random.choice(['like', 'attend', 'share'], n_events),
            'start_time': [(datetime.now() - timedelta(days=random.randint(30, 500))).strftime('%Y-%m-%d %H:%M:%S') for _ in range(n_events)],
            'city': np.random.choice(['Jakarta', 'Medan', 'Bandung', 'Surabaya', 'Singapore'], n_events),
            'lat': np.random.uniform(-10, 15, n_events),
            'lng': np.random.uniform(95, 120, n_events),
            'weather_description': np.random.choice(['sunny', 'cloudy', 'rainy'], n_events),
            'category': np.random.choice(EVENT_CATEGORIES, n_events)  # Use EVENT_CATEGORIES
        })

    # Extract locations from events data
    locations = extract_locations_from_events(events_df)

    # Generate synthetic user data based on event locations
    print("Generating synthetic user data based on event locations...")
    users_df = generate_synthetic_users(n=5000, locations=locations)

    # Assign users to events
    print("Assigning users to events...")
    events_with_users = assign_users_to_events(events_df, users_df)

    # Save results
    events_with_users.to_csv('events_with_users.csv', index=False)
    print("Data generation complete. Results saved to 'events_with_users.csv'")

if __name__ == "__main__":
    main()


Loaded 10 events
Extracted 6 distinct locations from event data
Generating synthetic user data based on event locations...
Assigning users to events...
Processing event 0 of 10
Data generation complete. Results saved to 'events_with_users.csv'


In [56]:
pd.read_csv("events_with_users.csv")

,Unnamed: 0,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,...,event_type,user_id,gender,joinedAt,location,age,lat_user,lng_user,user_distance_km_from_event,interaction_id
0,0,2587616435,2012-11-13 11:00:00.002000+00:00,Sihanoukville,10.633,103.500,93,65,317,47,...,Indoor,3611312262,female,2012-10-29 22:23:09+00:00,Sihanoukville,26,10.617021,103.491766,1.995357,2587616435_3611312262
1,1,1145166049,2013-07-08 02:00:00+00:00,Palo Alto,37.442,-122.172,162,112,1021,150,...,Indoor,81015721316,female,2012-10-13 14:28:07+00:00,Palo Alto,30,37.402608,-122.154158,4.800149,1145166049_81015721316
2,2,920600431,2012-07-29 19:00:00+00:00,Karachi,24.893,67.028,22,8,993,26,...,Indoor,20883410240,male,2012-09-15 21:16:26+00:00,Karachi,23,24.911830,67.028069,2.090134,920600431_20883410240
3,3,3580637647,2012-10-22 10:00:00.003000+00:00,Los Angeles,3.156,101.612,63,42,430,20,...,Indoor,6836768674,female,2012-10-18 14:37:05+00:00,Los Angeles,28,3.157339,101.619985,0.898675,3580637647_6836768674
4,4,1924180022,2012-11-11 20:00:00.003000+00:00,Palo Alto,37.416,-122.152,41,12,53,1,...,Indoor,51970286096,female,2012-09-22 13:49:23+00:00,Palo Alto,30,37.417645,-122.172800,2.316000,1924180022_51970286096
5,5,2396566946,2012-12-12 18:00:00.002000+00:00,Sihanoukville,10.633,103.500,391,275,1145,210,...,Indoor,92656282674,male,2012-10-07 22:05:04+00:00,Sihanoukville,25,10.623547,103.488041,1.692130,2396566946_92656282674
6,6,2654661184,2012-11-26 00:00:00.001000+00:00,Palo Alto,37.416,-122.152,7,6,16,1,...,Outdoor,43231362050,male,2012-09-27 21:06:16+00:00,Palo Alto,17,37.434130,-122.175199,3.268157,2654661184_43231362050
7,7,229252687,2012-09-30 00:00:00.001000+00:00,Palo Alto,37.416,-122.152,155,102,767,246,...,Outdoor,38260741567,male,2012-09-13 06:42:13+00:00,Palo Alto,27,37.405663,-122.163270,1.697453,229252687_38260741567
8,8,642644408,2012-10-31 00:00:00.001000+00:00,Siantar,2.965,99.075,283,111,3371,187,...,Indoor,33077088896,female,2012-10-16 15:47:04+00:00,Siantar,21,2.952967,99.078555,1.392699,642644408_33077088896
9,9,4032767837,2012-11-30 16:00:00.003000+00:00,Jakarta,-6.262,106.837,31,29,283,21,...,Indoor,51150426587,male,2012-09-14 12:36:02+00:00,Jakarta,22,-6.263284,106.845923,1.000608,4032767837_51150426587


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from sklearn.cluster import KMeans

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

EVENT_CATEGORIES = [
    'Music & Concerts',
    'Food & Drink',
    'Education & Learning',
    'Sports & Fitness',
    'Arts & Culture',
    'Business & Networking',
    'Technology',
    'Community & Causes',
    'Health & Wellness',
    'Entertainment',
    'Seasonal & Festivals',
    'Immersive Experiences'
]

def extract_locations_from_events(events_df, num_clusters=300):
    """
    Extract location information from events data and cluster
    into representative locations for user generation

    Parameters:
    -----------
    events_df : DataFrame
        Events dataframe with lat/lng coordinates
    num_clusters : int
        Number of location clusters to create

    Returns:
    --------
    List of tuples (location_name, lat, lng) to use for user generation
    """
    # Filter events with valid coordinates
    valid_events = events_df.dropna(subset=['lat', 'lng'])

    # If city column exists, use it
    if 'city' in valid_events.columns:
        # Group by city and get average coordinates
        city_groups = valid_events.groupby('city').agg({
            'lat': 'mean',
            'lng': 'mean'
        }).reset_index()

        # Convert to list of tuples
        locations = [(city, lat, lng) for city, lat, lng in
                     zip(city_groups['city'], city_groups['lat'], city_groups['lng'])]
    else:
        # Use KMeans clustering to identify geographic clusters
        coords = valid_events[['lat', 'lng']].values

        # Adjust num_clusters if we have fewer data points than requested clusters
        actual_clusters = min(num_clusters, len(coords))

        if actual_clusters <= 1:
            # Handle edge case with very little data
            return [("Default Location", coords[0][0], coords[0][1])]

        # Apply KMeans clustering
        kmeans = KMeans(n_clusters=actual_clusters, random_state=42, n_init=10)  # Added n_init
        kmeans.fit(coords)

        # Get cluster centers
        centers = kmeans.cluster_centers_

        # Create location names based on cluster ID
        locations = [(f"Cluster_{i}", center[0], center[1])
                     for i, center in enumerate(centers)]

    print(f"Extracted {len(locations)} distinct locations from event data")
    return locations

def generate_synthetic_users(n=66600, locations=None, gender_distribution={'male': 0.55, 'female': 0.45}, 
                             mean_age=35, age_std=15):
    if locations is None or len(locations) == 0:
        locations = [
            ("Jakarta", -6.200, 106.816),
            ("Singapore", 1.352, 103.820),
            ("New York", 40.712, -74.006),
            ("London", 51.507, -0.127)
        ]
    
    nearby_cities = {
        "Jakarta": ["Bekasi", "Tangerang", "Depok"],
        "Singapore": ["Johor Bahru", "Batam", "Bintan"],
        "New York": ["Jersey City", "Newark", "Yonkers"],
        "London": ["Greenwich", "Croydon", "Watford"]
    }
    
    user_ids, genders, joined_ats, user_locations, ages, lats, lngs = [], [], [], [], [], [], []
    
    start_date = datetime(2012, 9, 1)
    end_date = datetime(2012, 10, 31)
    
    gender_choices = np.random.choice(list(gender_distribution.keys()), size=n, p=list(gender_distribution.values()))
    location_weights = np.ones(len(locations)) / len(locations)
    
    for i in range(n):
        user_id = str(random.randint(10000000, 99999999999))
        gender = gender_choices[i]
        
        join_date = start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
        joined_at = join_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
        
        location_idx = np.random.choice(range(len(locations)), p=location_weights)
        location_name, base_lat, base_lng = locations[location_idx]
        
        if location_name in nearby_cities and random.random() < 0.3:  # 30% chance of nearby city
            location_name = random.choice(nearby_cities[location_name])
        
        variation_scale = 0.22  # About 24km at the equator
        lat = base_lat + random.uniform(-variation_scale, variation_scale)
        lng = base_lng + random.uniform(-variation_scale, variation_scale)
        
        if random.random() < 0.05:
            lat, lng = np.nan, np.nan
        
        age = random.randint(16, 100)  # Age range from 16 to 100
        
        user_ids.append(user_id)
        genders.append(gender)
        joined_ats.append(joined_at)
        user_locations.append(location_name)
        ages.append(age)
        lats.append(lat)
        lngs.append(lng)
    
    users_df = pd.DataFrame({
        'user_id': user_ids,
        'gender': genders,
        'joinedAt': joined_ats,
        'location': user_locations,
        'age': ages,
        'lat': lats,
        'lng': lngs
    })
    
    return users_df


def assign_users_to_events(events_df, users_df, bias_local=True, bias_category=True):
    """
    Assign users to events with realistic biases

    Parameters:
    -----------
    events_df : DataFrame
        Events dataframe
    users_df : DataFrame
        Users dataframe
    bias_local : bool
        Whether to bias users toward nearby events
    bias_category : bool
        Whether to bias users toward certain categories based on demographics

    Returns:
    --------
    Updated events dataframe with user information
    """
    # Create copy of events dataframe
    events_with_users = events_df.copy()

    # Ensure lat/lng are numeric in both dataframes
    for df in [events_with_users, users_df]:
        for col in ['lat', 'lng']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

    # Convert 'start_time' and 'joinedAt' to datetime objects
    if 'start_time' in events_with_users.columns:
        events_with_users['start_time'] = pd.to_datetime(events_with_users['start_time'], errors='coerce')
    users_df['joinedAt'] = pd.to_datetime(users_df['joinedAt'], errors='coerce')

    # Define category biases
    category_biases = {
        'Music & Concerts': {'age_range': (16, 35), 'gender_bias': None},
        'Sports & Fitness': {'age_range': (16, 45), 'gender_bias': 'male'},
        'Arts & Culture': {'age_range': (25, 55), 'gender_bias': 'female'},
        'Technology': {'age_range': (16, 40), 'gender_bias': None},
        'Food & Drink': {'age_range': (16, 100), 'gender_bias': None},
        'Business & Networking': {'age_range': (30, 60), 'gender_bias': None},
        'Education & Learning': {'age_range': (16, 50), 'gender_bias': None},
        'Community & Causes': {'age_range': (16, 100), 'gender_bias': None},
        'Health & Wellness':  {'age_range': (16, 100), 'gender_bias': 'female'},
        'Entertainment': {'age_range': (16, 100), 'gender_bias': None},
        'Seasonal & Festivals': {'age_range': (16, 100), 'gender_bias': None},
        'Immersive Experiences': {'age_range': (16, 100), 'gender_bias': None},
    }

    # Add user information to events
    user_cols = ['user_id', 'gender', 'joinedAt', 'location', 'age', 'lat_user', 'lng_user']
    for col in user_cols:
        events_with_users[col] = None  # Initialize columns

    events_with_users['user_distance_km_from_event'] = None
    events_with_users['interaction_id'] = None
    events_with_users['lat_event'] = None #initialise events lat
    events_with_users['lng_event'] = None #initialise event lng

    for idx, event in events_with_users.iterrows():
        if idx % 1000 == 0:
            print(f"Processing event {idx} of {len(events_with_users)}")

        # Filter eligible users based on join date
        if 'start_time' in event and not pd.isna(event['start_time']):
            event_date = event['start_time'].date()
            eligible_users = users_df[users_df['joinedAt'].dt.date < event_date]
        else:
            eligible_users = users_df

        if len(eligible_users) == 0:
            eligible_users = users_df

        # Apply category bias
        if bias_category and 'category' in event and not pd.isna(event['category']):
            category = event['category']  # Use the actual category names
            if category in category_biases:
                bias = category_biases[category]
                if bias['age_range']:
                    min_age, max_age = bias['age_range']
                    eligible_users = eligible_users[(eligible_users['age'] >= min_age) & (eligible_users['age'] <= max_age)]
                if bias['gender_bias']:
                    eligible_users = eligible_users[eligible_users['gender'] == bias['gender_bias']]

        # Apply geographic bias
        if bias_local and 'lat' in event and 'lng' in event and not pd.isna(event['lat']) and not pd.isna(event['lng']):
            valid_users = eligible_users.dropna(subset=['lat', 'lng']).copy()
            if not valid_users.empty:
                valid_users['distance'] = valid_users.apply(
                    lambda user: np.sqrt((user['lat'] - event['lat'])**2 + (user['lng'] - event['lng'])**2),
                    axis=1
                )
                weights = 1 / (valid_users['distance'] + 0.1)
                weights = weights / weights.sum()
                selected_user = valid_users.sample(n=1, weights=weights).iloc[0]
                distance_km = selected_user['distance'] * 111.0
            else:
                selected_user = eligible_users.sample(n=1).iloc[0]
                distance_km = None
        else:
            selected_user = eligible_users.sample(n=1).iloc[0]
            distance_km = None

        # Assign user data to event
        events_with_users.at[idx, 'user_id'] = selected_user['user_id']
        events_with_users.at[idx, 'gender'] = selected_user['gender']
        events_with_users.at[idx, 'joinedAt'] = selected_user['joinedAt']
        events_with_users.at[idx, 'location'] = selected_user['location']
        events_with_users.at[idx, 'age'] = selected_user['age']
        events_with_users.at[idx, 'lat_user'] = selected_user['lat']
        events_with_users.at[idx, 'lng_user'] = selected_user['lng']
        events_with_users.at[idx, 'user_distance_km_from_event'] = distance_km

        # Create interaction_id
        events_with_users.at[idx, 'interaction_id'] = f"{event['event_id']}_{selected_user['user_id']}"


    return events_with_users


def main():
    # Load original events data
    try:
        events_df = pd.read_csv("users_2_gen.csv")
        events_df = events_df.iloc[:10]
        print(f"Loaded {len(events_df)} events")
        
        # Count unique events
        unique_events = events_df['event_id'].nunique()
        print(f"Number of unique events: {unique_events}")
        
        # Calculate the number of users needed (at least 10 per unique event)
        num_users = max(66600, unique_events * 10)
        
        # Extract locations from events data
        locations = extract_locations_from_events(events_df)

        # Generate synthetic user data based on event locations
        print(f"Generating {num_users} synthetic users based on event locations...")
        users_df = generate_synthetic_users(n=num_users, locations=locations)

        # Assign users to events
        print("Assigning users to events...")
        events_with_users = assign_users_to_events(events_df, users_df)

        # Save results
        events_with_users.to_csv('events_with_users.csv', index=False)
        print("Data generation complete. Results saved to 'events_with_users.csv'")
        
    except FileNotFoundError:
        print("Events data file not found. Please ensure 'users_2_gen.csv' is in the correct directory.")

if __name__ == "__main__":
    main()


Loaded 10 events
Number of unique events: 2
Extracted 2 distinct locations from event data
Generating 66600 synthetic users based on event locations...
Assigning users to events...
Processing event 0 of 10
Data generation complete. Results saved to 'events_with_users.csv'


In [2]:
events_with_users = pd.read_csv("events_with_users.csv")
events_with_users

,Unnamed: 0,event_id,user_id,interaction_type,start_time,city,lat,lng,weather_description,category,...,gender,joinedAt,location,age,lat_user,lng_user,user_distance_km_from_event,interaction_id,lat_event,lng_event
0,0,1186208412,83764459012,maybe,2012-08-08 12:00:00+00:00,Toronto,43.784,-79.187,Cloudy,Education & Learning,...,male,2012-09-30 04:55:53+00:00,Toronto,18,43.643841,-79.144825,16.246711,1186208412_83764459012,43.784,-79.187
1,1,1186208412,60244156011,maybe,2012-08-08 12:00:00+00:00,Toronto,43.784,-79.187,Cloudy,Education & Learning,...,female,2012-09-16 06:12:33+00:00,Toronto,41,43.942272,-79.295556,21.303431,1186208412_60244156011,43.784,-79.187
2,2,1186208412,45891665893,invited & maybe,2012-08-08 12:00:00+00:00,Toronto,43.784,-79.187,Cloudy,Education & Learning,...,male,2012-09-28 21:13:46+00:00,Toronto,38,43.679051,-79.289959,16.319144,1186208412_45891665893,43.784,-79.187
3,3,1186208412,36810626350,invited & maybe,2012-08-08 12:00:00+00:00,Toronto,43.784,-79.187,Cloudy,Education & Learning,...,female,2012-09-12 01:27:25+00:00,Toronto,16,43.817362,-79.284735,11.463253,1186208412_36810626350,43.784,-79.187
4,4,1186208412,88571258779,no,2012-08-08 12:00:00+00:00,Toronto,43.784,-79.187,Cloudy,Education & Learning,...,male,2012-09-08 16:46:35+00:00,Toronto,17,43.883270,-79.043762,19.344537,1186208412_88571258779,43.784,-79.187
5,5,1186208412,38061609239,no,2012-08-08 12:00:00+00:00,Toronto,43.784,-79.187,Cloudy,Education & Learning,...,female,2012-10-05 01:50:29+00:00,Toronto,32,43.786848,-79.170968,1.807430,1186208412_38061609239,43.784,-79.187
6,6,855842686,23512539333,yes,2012-12-14 04:00:00.003000+00:00,Kamloops,50.676,-120.334,Snow,Technology,...,male,2012-10-12 21:38:47+00:00,Kamloops,19,50.807074,-120.169552,23.342643,855842686_23512539333,50.676,-120.334
7,7,855842686,69603336646,yes,2012-12-14 04:00:00.003000+00:00,Kamloops,50.676,-120.334,Snow,Technology,...,female,2012-09-21 20:55:11+00:00,Kamloops,23,50.895999,-120.526051,32.415619,855842686_69603336646,50.676,-120.334
8,8,855842686,15660045845,yes,2012-12-14 04:00:00.003000+00:00,Kamloops,50.676,-120.334,Snow,Technology,...,male,2012-10-10 11:03:18+00:00,Kamloops,23,50.483106,-120.202370,25.921458,855842686_15660045845,50.676,-120.334
9,9,855842686,8601024617,yes,2012-12-14 04:00:00.003000+00:00,Kamloops,50.676,-120.334,Snow,Technology,...,male,2012-09-15 08:12:23+00:00,Kamloops,35,50.772365,-120.231838,15.588776,855842686_8601024617,50.676,-120.334


In [65]:
events_with_users["user_distance_km_from_event"].max()
output:
np.float64(873.0335899824856)

np.float64(873.0335899824856)

In [61]:
print(len(events_with_users))
events_with_users["age"].max()


10


np.int64(81)

In [51]:
pd.read_csv("synthetic_users.csv").head(10)

,user_id,gender,joinedAt,location,age,lat,lng
0,15641219101,male,2012-09-03 10:16:45+00:00,Los Angeles,27,3.165662,101.601796
1,90644526631,female,2012-10-23 22:47:46+00:00,Los Angeles,21,NaN,NaN
2,31013814027,female,2012-10-20 01:33:47+00:00,Sihanoukville,22,10.637081,103.502450
3,76036611852,female,2012-10-11 17:33:07+00:00,Sihanoukville,21,10.621818,103.503571
4,24945803837,male,2012-10-12 00:45:46+00:00,Karachi,16,24.886610,67.014219
5,14340564473,male,2012-09-10 00:06:48+00:00,Siantar,20,2.960197,99.069359
6,98980866019,male,2012-10-15 14:32:15+00:00,Palo Alto,16,37.423949,-122.138075
7,41035702129,female,2012-10-06 02:40:14+00:00,Karachi,24,NaN,NaN
8,51983405392,female,2012-09-27 23:43:31+00:00,Jakarta,25,-6.263864,106.850364
9,27305679827,female,2012-09-26 22:06:12+00:00,Sihanoukville,16,10.641073,103.507342


In [50]:
pd.read_csv("/home/nkama/masters_thesis_project/thesis/events_with_users.csv").head(10)

,Unnamed: 0,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_description,category,title,event_type,user_id
0,0,2587616435,2012-11-13 11:00:00.002000+00:00,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,3611312262
1,1,1145166049,2013-07-08 02:00:00+00:00,Palo Alto,37.442,-122.172,162,112,1021,150,1445,Cloudy,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor,7283896301
2,2,920600431,2012-07-29 19:00:00+00:00,Karachi,24.893,67.028,22,8,993,26,1049,Cloudy,Entertainment,Karachi Nights: Summer Fiesta Under the Stars,Indoor,20883410240
3,3,3580637647,2012-10-22 10:00:00.003000+00:00,Los Angeles,3.156,101.612,63,42,430,20,555,Light Rain,Arts & Culture,Funky Art Walk LA - Morning Vibes & Visual Del...,Indoor,51369360078
4,4,1924180022,2012-11-11 20:00:00.003000+00:00,Palo Alto,37.416,-122.152,41,12,53,1,107,Cloudy,Arts & Culture,Palo Alto Art After Dark - A Fall Fiesta of Cr...,Indoor,29303938484
5,5,2396566946,2012-12-12 18:00:00.002000+00:00,Sihanoukville,10.633,103.500,391,275,1145,210,2021,Light Drizzle,Technology,Winter Tech Fest Sihanoukville 2025 - Innovate...,Indoor,84854421696
6,6,2654661184,2012-11-26 00:00:00.001000+00:00,Palo Alto,37.416,-122.152,7,6,16,1,30,Partly Cloudy,Seasonal & Festivals,Palo Alto Harvest Moon Lights Festival 2025,Outdoor,43231362050
7,7,229252687,2012-09-30 00:00:00.001000+00:00,Palo Alto,37.416,-122.152,155,102,767,246,1270,Partly Cloudy,Immersive Experiences,Palo Alto Nightscape - Immersive Experience Un...,Outdoor,38260741567
8,8,642644408,2012-10-31 00:00:00.001000+00:00,Siantar,2.965,99.075,283,111,3371,187,3952,Rain,Technology,"Siantar Tech Night Ignite 2025 - Innovate, Con...",Indoor,8572219167
9,9,4032767837,2012-11-30 16:00:00.003000+00:00,Jakarta,-6.262,106.837,31,29,283,21,364,Rain,Education & Learning,Empower Jakarta - Spring Learning Odyssey,Indoor,51150426587


In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker
import uuid

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
fake = Faker()
Faker.seed(42)

def generate_synthetic_users(n=5000, gender_distribution={'male': 0.55, 'female': 0.45}, 
                          mean_age=22, age_std=5):
    """
    Generate synthetic user data
    
    Parameters:
    -----------
    n : int
        Number of users to generate
    gender_distribution : dict
        Distribution of genders
    mean_age : int
        Mean age of users
    age_std : int
        Standard deviation of age
        
    Returns:
    --------
    DataFrame with synthetic user data
    """
    # Generate user IDs
    user_ids = [str(uuid.uuid4()) for _ in range(n)]
    
    # Generate genders based on distribution
    genders = np.random.choice(
        list(gender_distribution.keys()), 
        size=n, 
        p=list(gender_distribution.values())
    )
    
    # Generate ages from normal distribution, bounded between 18 and 65
    ages = np.random.normal(mean_age, age_std, n)
    ages = np.clip(ages, 18, 65).astype(int)
    
    # Generate join dates (between 1 and 3 years ago)
    now = datetime.now()
    join_dates = [now - timedelta(days=random.randint(30, 1095)) for _ in range(n)]
    join_dates = [d.strftime('%Y-%m-%d') for d in join_dates]
    
    # Generate locations and coordinates
    locations = []
    lats = []
    lngs = []
    
    for _ in range(n):
        # Use faker to generate realistic locations
        location = fake.city() + ', ' + fake.state_abbr()
        locations.append(location)
        
        # Generate random coordinates (roughly US)
        lat = np.random.uniform(25, 49)  # US latitude range approximately
        lng = np.random.uniform(-125, -65)  # US longitude range approximately
        lats.append(lat)
        lngs.append(lng)
    
    # Create dataframe
    users_df = pd.DataFrame({
        'user_id': user_ids,
        'gender': genders,
        'joinedAt': join_dates,
        'location': locations,
        'age': ages,
        'lat': lats,
        'lng': lngs
    })
    
    return users_df

def assign_users_to_events(events_df, users_df, bias_local=True, bias_category=True):
    """
    Assign users to events with realistic biases
    
    Parameters:
    -----------
    events_df : DataFrame
        Events dataframe
    users_df : DataFrame
        Users dataframe
    bias_local : bool
        Whether to bias users toward nearby events
    bias_category : bool
        Whether to bias users toward certain categories based on demographics
        
    Returns:
    --------
    Updated events dataframe with user_id column
    """
    # Create copy of events dataframe
    events_with_users = events_df.copy()
    
    # List to store user assignments
    user_assignments = []
    
    # Track how many events each user has attended
    user_event_count = {user_id: 0 for user_id in users_df['user_id']}
    
    # Define category preferences (simplified example)
    category_biases = {
        'music': {'age_range': (18, 35), 'gender_bias': None},
        'sports': {'age_range': (18, 45), 'gender_bias': 'male'},
        'art': {'age_range': (25, 55), 'gender_bias': 'female'},
        'technology': {'age_range': (20, 40), 'gender_bias': None},
        'food': {'age_range': None, 'gender_bias': None},
        'business': {'age_range': (30, 60), 'gender_bias': None},
        'education': {'age_range': (20, 50), 'gender_bias': None},
    }
    
    # Process each event
    for idx, event in events_df.iterrows():
        # Determine eligible users based on join date and event date
        event_date = pd.to_datetime(event['start_time']).date()
        eligible_users = users_df[pd.to_datetime(users_df['joinedAt']).dt.date < event_date]
        
        if len(eligible_users) == 0:
            # Use any user if no eligible users (edge case)
            eligible_users = users_df
        
        # Apply category bias if enabled
        if bias_category and event['category'].lower() in category_biases:
            bias = category_biases[event['category'].lower()]
            
            # Filter by age range if specified
            if bias['age_range']:
                min_age, max_age = bias['age_range']
                eligible_users = eligible_users[(eligible_users['age'] >= min_age) & 
                                             (eligible_users['age'] <= max_age)]
            
            # Filter by gender if specified
            if bias['gender_bias']:
                gender_users = eligible_users[eligible_users['gender'] == bias['gender_bias']]
                # Only apply gender filter if we don't end up with empty set
                if len(gender_users) > 0:
                    eligible_users = gender_users
        
        # Apply geographic bias if enabled
        if bias_local and not pd.isna(event['lat']) and not pd.isna(event['lng']):
            # Calculate distance from event to each user
            eligible_users['distance'] = eligible_users.apply(
                lambda user: np.sqrt((user['lat'] - event['lat'])**2 + 
                                   (user['lng'] - event['lng'])**2),
                axis=1
            )
            
            # Weight users by inverse distance (closer = higher probability)
            weights = 1 / (eligible_users['distance'] + 0.1)  # Add small constant to avoid division by zero
            weights = weights / weights.sum()
            
            # Sample user with distance bias
            selected_user = eligible_users.sample(n=1, weights=weights)['user_id'].iloc[0]
        else:
            # Random selection if no geographic bias or missing coordinates
            selected_user = eligible_users.sample(n=1)['user_id'].iloc[0]
        
        # Update user event count and add to assignments
        user_event_count[selected_user] += 1
        user_assignments.append(selected_user)
    
    # Add user assignments to events dataframe
    events_with_users['user_id'] = user_assignments
    
    # Print distribution statistics
    events_per_user = pd.Series(user_event_count).value_counts()
    print(f"Events per user distribution:\n{events_per_user.describe()}")
    print(f"Users with no events: {sum(count == 0 for count in user_event_count.values())}")
    
    return events_with_users

def main():
    # Load original events data (assuming it exists)
    # Replace with your actual loading logic
    try:
        df = events_df = pd.read_csv("complete_events_data.csv")
        events_df = df.iloc[:10]
    except FileNotFoundError:
        # Generate sample events data for demonstration
        print("Creating sample events data for demonstration")
        events_df = pd.DataFrame({
            'event_id': range(1000),
            'interaction_type': np.random.choice(['like', 'attend', 'share'], 1000),
            'start_time': [datetime.now() - timedelta(days=random.randint(1, 500)) for _ in range(1000)],
            'city': [fake.city() for _ in range(1000)],
            'lat': np.random.uniform(25, 49, 1000),
            'lng': np.random.uniform(-125, -65, 1000),
            'weather_description': np.random.choice(['sunny', 'cloudy', 'rainy'], 1000),
            'category': np.random.choice(['music', 'sports', 'art', 'technology', 'food', 'business', 'education'], 1000),
            'title': [f"Event {i}" for i in range(1000)],
            'event_type': np.random.choice(['concert', 'meeting', 'class', 'exhibition'], 1000)
        })
    
    # Generate synthetic users
    print("Generating synthetic user data...")
    users_df = generate_synthetic_users(n=5000, gender_distribution={'male': 0.55, 'female': 0.45}, mean_age=22)
    
    # Assign users to events
    print("Assigning users to events...")
    events_with_users = assign_users_to_events(events_df, users_df)
    
    # Save results
    users_df.to_csv('synthetic_users.csv', index=False)
    events_with_users.to_csv('events_with_users.csv', index=False)
    
    print(f"Generated {len(users_df)} users")
    print(f"Assigned users to {len(events_with_users)} events")
    print("Files saved: synthetic_users.csv, events_with_users.csv")
    
    # Return dataframes for further use
    return users_df, events_with_users

if __name__ == "__main__":
    main()

Generating synthetic user data...
Assigning users to events...


/tmp/ipykernel_150178/3640833777.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_users['distance'] = eligible_users.apply(


Events per user distribution:
count       2.00000
mean     2500.00000
std      3521.39177
min        10.00000
25%      1255.00000
50%      2500.00000
75%      3745.00000
max      4990.00000
Name: count, dtype: float64
Users with no events: 4990
Generated 5000 users
Assigned users to 10 events
Files saved: synthetic_users.csv, events_with_users.csv


/tmp/ipykernel_150178/3640833777.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_users['distance'] = eligible_users.apply(


In [21]:
pd.read_csv("/home/nkama/masters_thesis_project/thesis/user_event_pairs.csv")

,Unnamed: 0,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_description,category,title,event_type,user_id
0,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,092f87a3-23fb-4fc9-b1c0-7f7cd10cc499
1,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,a6966474-f80e-4583-983a-3e8a4bec1aa6
2,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,21493c35-907b-494f-8ce3-b325a609bf04
3,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,c14506e7-5b31-4b9c-a61e-3529c1f3dacd
4,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,9aafe453-fa9c-463b-b093-4b4b8b4777c7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,9,4032767837,2012-11-30T16:00:00.003Z,Jakarta,-6.262,106.837,31,29,283,21,364,Rain,Education & Learning,Empower Jakarta - Spring Learning Odyssey,Indoor,199f2d38-67d6-493b-bbb0-3434d67c4308
257,9,4032767837,2012-11-30T16:00:00.003Z,Jakarta,-6.262,106.837,31,29,283,21,364,Rain,Education & Learning,Empower Jakarta - Spring Learning Odyssey,Indoor,7d3e8fe0-9367-43a0-802e-f8d74c224c8d
258,9,4032767837,2012-11-30T16:00:00.003Z,Jakarta,-6.262,106.837,31,29,283,21,364,Rain,Education & Learning,Empower Jakarta - Spring Learning Odyssey,Indoor,e0bbf2a7-7e41-46cd-8485-b1f2ba4fd9ce
259,9,4032767837,2012-11-30T16:00:00.003Z,Jakarta,-6.262,106.837,31,29,283,21,364,Rain,Education & Learning,Empower Jakarta - Spring Learning Odyssey,Indoor,b7ae6a85-0dfa-4458-a4e9-ac220ac533f8


In [14]:
pd.read_csv("/home/nkama/masters_thesis_project/thesis/events_with_users.csv")


,Unnamed: 0,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_description,category,title,event_type,user_id
0,0,2587616435,2012-11-13T11:00:00.002Z,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,07af768a-b836-469b-a667-b274c7c8d6cc
1,1,1145166049,2013-07-08T02:00:00.000Z,Palo Alto,37.442,-122.172,162,112,1021,150,1445,Cloudy,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor,2900a1c3-138c-46a0-9948-5c0c1cf7c677
2,2,920600431,2012-07-29T19:00:00.000Z,Karachi,24.893,67.028,22,8,993,26,1049,Cloudy,Entertainment,Karachi Nights: Summer Fiesta Under the Stars,Indoor,116821ed-302d-4b79-a082-fcb5c4dac3b0
3,3,3580637647,2012-10-22T10:00:00.003Z,Los Angeles,3.156,101.612,63,42,430,20,555,Light Rain,Arts & Culture,Funky Art Walk LA - Morning Vibes & Visual Del...,Indoor,2f2ae645-1257-48ba-804f-71f9c5b8d923
4,4,1924180022,2012-11-11T20:00:00.003Z,Palo Alto,37.416,-122.152,41,12,53,1,107,Cloudy,Arts & Culture,Palo Alto Art After Dark - A Fall Fiesta of Cr...,Indoor,3535c803-b368-4c66-ba2d-3d12216ce3ac
5,5,2396566946,2012-12-12T18:00:00.002Z,Sihanoukville,10.633,103.500,391,275,1145,210,2021,Light Drizzle,Technology,Winter Tech Fest Sihanoukville 2025 - Innovate...,Indoor,c00cfec6-2573-4292-8685-149d9c749a0a
6,6,2654661184,2012-11-26T00:00:00.001Z,Palo Alto,37.416,-122.152,7,6,16,1,30,Partly Cloudy,Seasonal & Festivals,Palo Alto Harvest Moon Lights Festival 2025,Outdoor,108427bc-3cdc-4990-8771-e508e5358b2c
7,7,229252687,2012-09-30T00:00:00.001Z,Palo Alto,37.416,-122.152,155,102,767,246,1270,Partly Cloudy,Immersive Experiences,Palo Alto Nightscape - Immersive Experience Un...,Outdoor,c62b801b-346e-46e6-8a69-78a6569b6628
8,8,642644408,2012-10-31T00:00:00.001Z,Siantar,2.965,99.075,283,111,3371,187,3952,Rain,Technology,"Siantar Tech Night Ignite 2025 - Innovate, Con...",Indoor,e4e5f932-1e06-451b-a6d5-9d8a134eb529
9,9,4032767837,2012-11-30T16:00:00.003Z,Jakarta,-6.262,106.837,31,29,283,21,364,Rain,Education & Learning,Empower Jakarta - Spring Learning Odyssey,Indoor,ebd4e0a8-f3a5-4a97-a7e3-c6e62988605e


In [24]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

def generate_synthetic_users(n=5000, gender_distribution={'male': 0.55, 'female': 0.45}, 
                          mean_age=22, age_std=5):
    """
    Generate synthetic user data
    
    Parameters:
    -----------
    n : int
        Number of users to generate
    gender_distribution : dict
        Distribution of genders
    mean_age : int
        Mean age of users
    age_std : int
        Standard deviation of age
        
    Returns:
    --------
    DataFrame with synthetic user data
    """
    # List of cities with their coordinates (focusing on Indonesian cities)
    cities = [
        ("Medan", 3.590, 98.674),
        ("Jakarta", -6.200, 106.816),
        ("Djokja Yogyakarta", -7.797, 110.370),
        ("Surabaya", -7.250, 112.750),
        ("Bandung", -6.914, 107.609),
        ("Makassar", -5.147, 119.432),
        ("Denpasar", -8.650, 115.216),
        ("Palembang", -2.976, 104.774),
        ("Singapore", 1.352, 103.820),
        ("Kuala Lumpur", 3.139, 101.687),
        ("Bangkok", 13.736, 100.523),
        ("Manila", 14.599, 120.984),
        ("Triolet", -20.057, 57.552)
    ]
    
    # Initialize lists for each column
    user_ids = []
    genders = []
    joined_ats = []
    locations = []
    ages = []
    lats = []
    lngs = []
    
    # Generate join dates between 2012-09-01 and 2012-10-31
    start_date = datetime(2012, 9, 1)
    end_date = datetime(2012, 10, 31)
    
    # Generate genders based on distribution
    gender_choices = np.random.choice(
        list(gender_distribution.keys()), 
        size=n, 
        p=list(gender_distribution.values())
    )
    
    for i in range(n):
        # Generate a random numeric ID between 8 and 11 digits
        user_id = str(random.randint(10000000, 99999999999))
        
        # Get gender from pre-generated choices
        gender = gender_choices[i]
        
        # Generate random join date
        join_date = start_date + timedelta(
            seconds=random.randint(0, int((end_date - start_date).total_seconds()))
        )
        joined_at = join_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
        
        # Select random city and coordinates
        city, lat, lng = random.choice(cities)
        
        # Sometimes add country name to location (for variety)
        if random.random() < 0.3:
            if "Indonesia" in city:
                location = city
            else:
                location = f"{city} Indonesia"
        else:
            location = city
        
        # Add small random variations to coordinates
        lat += random.uniform(-0.01, 0.01)
        lng += random.uniform(-0.01, 0.01)
        
        # Occasionally set coordinates to NaN (about 5% of the time)
        if random.random() < 0.05:
            lat, lng = np.nan, np.nan
        
        # Generate age with a normal distribution around mean_age
        age = int(np.random.normal(mean_age, age_std))
        age = max(16, min(60, age))  # Clip between 16 and 60
        
        # Append to lists
        user_ids.append(user_id)
        genders.append(gender)
        joined_ats.append(joined_at)
        locations.append(location)
        ages.append(age)
        lats.append(lat)
        lngs.append(lng)
    
    # Create dataframe
    users_df = pd.DataFrame({
        'user_id': user_ids,
        'gender': genders,
        'joinedAt': joined_ats,
        'location': locations,
        'age': ages,
        'lat': lats,
        'lng': lngs
    })
    
    return users_df

def assign_users_to_events(events_df, users_df, bias_local=True, bias_category=True):
    """
    Assign users to events with realistic biases
    
    Parameters:
    -----------
    events_df : DataFrame
        Events dataframe
    users_df : DataFrame
        Users dataframe
    bias_local : bool
        Whether to bias users toward nearby events
    bias_category : bool
        Whether to bias users toward certain categories based on demographics
        
    Returns:
    --------
    Updated events dataframe with user_id column
    """
    # Create copy of events dataframe
    events_with_users = events_df.copy()
    
    # Convert coordinates to numeric if they aren't already
    for df in [events_with_users, users_df]:
        for col in ['lat', 'lng']:
            if df[col].dtype == 'object':
                df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Make sure timestamps are datetime objects
    if 'start_time' in events_with_users.columns:
        events_with_users['start_time'] = pd.to_datetime(events_with_users['start_time'])
    users_df['joinedAt'] = pd.to_datetime(users_df['joinedAt'])
    
    # List to store assigned user IDs
    assigned_users = []
    
    # Define category preferences (simplified example)
    category_biases = {
        'music': {'age_range': (18, 35), 'gender_bias': None},
        'sports': {'age_range': (18, 45), 'gender_bias': 'male'},
        'art': {'age_range': (25, 55), 'gender_bias': 'female'},
        'technology': {'age_range': (20, 40), 'gender_bias': None},
        'food': {'age_range': None, 'gender_bias': None},
        'business': {'age_range': (30, 60), 'gender_bias': None},
        'education': {'age_range': (20, 50), 'gender_bias': None},
    }
    
    # Track how many events each user has attended
    user_event_count = {user_id: 0 for user_id in users_df['user_id']}
    
    # Process each event
    for idx, event in events_with_users.iterrows():
        # Determine eligible users based on join date and event date
        if 'start_time' in event and not pd.isna(event['start_time']):
            event_date = pd.to_datetime(event['start_time']).date()
            eligible_users = users_df[pd.to_datetime(users_df['joinedAt']).dt.date < event_date]
        else:
            eligible_users = users_df  # If no start_time, all users are eligible

        if len(eligible_users) == 0:
            # Use any user if no eligible users (edge case)
            eligible_users = users_df
        
        # Apply category bias if enabled and category exists
        if bias_category and 'category' in event and not pd.isna(event['category']) and event['category'].lower() in category_biases:
            bias = category_biases[event['category'].lower()]
            
            # Filter by age range if specified
            if bias['age_range']:
                min_age, max_age = bias['age_range']
                filtered_users = eligible_users[(eligible_users['age'] >= min_age) & 
                                             (eligible_users['age'] <= max_age)]
                # Only apply age filter if we don't end up with empty set
                if len(filtered_users) > 0:
                    eligible_users = filtered_users
            
            # Filter by gender if specified
            if bias['gender_bias']:
                gender_users = eligible_users[eligible_users['gender'] == bias['gender_bias']]
                # Only apply gender filter if we don't end up with empty set
                if len(gender_users) > 0:
                    eligible_users = gender_users
        
        # Apply geographic bias if enabled and coordinates exist
        if bias_local and 'lat' in event and 'lng' in event and not pd.isna(event['lat']) and not pd.isna(event['lng']):
            # Filter out users with NaN coordinates
            valid_users = eligible_users.dropna(subset=['lat', 'lng'])
            
            if len(valid_users) > 0:
                # Calculate distance from event to each user
                valid_users = valid_users.copy()  # Create an explicit copy
                valid_users['distance'] = valid_users.apply(
                    lambda user: np.sqrt((user['lat'] - event['lat'])**2 + 
                                    (user['lng'] - event['lng'])**2),
                    axis=1
                )
                
                # Weight users by inverse distance (closer = higher probability)
                weights = 1 / (valid_users['distance'] + 0.1)  # Add small constant to avoid division by zero
                weights = weights / weights.sum()
                
                # Sample user with distance bias
                selected_user = valid_users.sample(n=1, weights=weights)['user_id'].iloc[0]
            else:
                # Random selection if no valid coordinates
                selected_user = eligible_users.sample(n=1)['user_id'].iloc[0]
        else:
            # Random selection if no geographic bias or missing coordinates
            selected_user = eligible_users.sample(n=1)['user_id'].iloc[0]
        
        # Update user event count and add to assignments
        user_event_count[selected_user] += 1
        assigned_users.append(selected_user)
    
    # Add user assignments to events dataframe
    events_with_users['user_id'] = assigned_users
    
    # Print distribution statistics
    events_per_user = pd.Series(user_event_count).value_counts()
    print(f"Events per user distribution:\n{events_per_user.describe()}")
    print(f"Users with no events: {sum(count == 0 for count in user_event_count.values())}")
    
    return events_with_users

def main():
    # Load original events data
    try:
        df = events_df = pd.read_csv("complete_events_data.csv")
        events_df = df.iloc[:10]
        print(f"Loaded {len(events_df)} events")
    except FileNotFoundError:
        print("Events data file not found. Creating sample data for demonstration.")
        # Generate sample data for demonstration
        n_events = 11627  # Match the number in the problem statement
        events_df = pd.DataFrame({
            'event_id': range(1, n_events + 1),
            'interaction_type': np.random.choice(['like', 'attend', 'share'], n_events),
            'start_time': [(datetime.now() - timedelta(days=random.randint(30, 500))).strftime('%Y-%m-%d %H:%M:%S') for _ in range(n_events)],
            'city': np.random.choice(['Jakarta', 'Medan', 'Bandung', 'Surabaya', 'Singapore'], n_events),
            'lat': np.random.uniform(-10, 15, n_events),
            'lng': np.random.uniform(95, 120, n_events),
            'weather_description': np.random.choice(['sunny', 'cloudy', 'rainy'], n_events),
            'category': np.random.choice(['music', 'sports', 'art', 'technology', 'food', 'business'], n_events),
            'title': [f"Event {i}" for i in range(1, n_events + 1)],
            'event_type': np.random.choice(['concert', 'meeting', 'class', 'exhibition'], n_events)
        })
    
    # Generate synthetic users
    print("Generating synthetic user data...")
    users_df = generate_synthetic_users(n=5000, gender_distribution={'male': 0.55, 'female': 0.45}, mean_age=22)
    
    # Print gender distribution
    gender_counts = users_df['gender'].value_counts()
    print(f"Gender distribution:\n{gender_counts}")
    
    # Assign users to events
    print("Assigning users to events...")
    events_with_users = assign_users_to_events(events_df, users_df)
    
    # Save results
    users_df.to_csv('synthetic_users.csv', index=False)
    events_with_users.to_csv('events_with_users.csv', index=False)
    
    print(f"Generated {len(users_df)} users")
    print(f"Assigned users to {len(events_with_users)} events")
    print("Files saved: synthetic_users.csv, events_with_users.csv")
    
    # Return dataframes for further use
    return users_df, events_with_users

if __name__ == "__main__":
    main()

Loaded 10 events
Generating synthetic user data...
Gender distribution:
gender
male      2748
female    2252
Name: count, dtype: int64
Assigning users to events...
Events per user distribution:
count       2.00000
mean     2500.00000
std      3521.39177
min        10.00000
25%      1255.00000
50%      2500.00000
75%      3745.00000
max      4990.00000
Name: count, dtype: float64
Users with no events: 4990
Generated 5000 users
Assigned users to 10 events
Files saved: synthetic_users.csv, events_with_users.csv


In [25]:
pd.read_csv("/home/nkama/masters_thesis_project/thesis/synthetic_users.csv")

,user_id,gender,joinedAt,location,age,lat,lng
0,15641219101,male,2012-09-03 10:16:45+00:00,Manila Indonesia,19,14.593464,120.988729
1,76856326096,female,2012-09-09 10:34:55+00:00,Kuala Lumpur,16,3.129596,101.681373
2,73138415155,female,2012-09-20 07:19:31+00:00,Manila,19,14.599899,120.978409
3,24945803837,female,2012-10-12 00:45:46+00:00,Makassar Indonesia,26,-5.152694,119.437270
4,14526677245,male,2012-10-05 20:28:57+00:00,Makassar,24,-5.140857,119.436595
...,...,...,...,...,...,...,...
4995,38764159728,female,2012-09-02 03:47:19+00:00,Makassar,33,-5.155448,119.432438
4996,90464797057,male,2012-10-04 06:35:23+00:00,Medan,24,3.595907,98.664053
4997,22858839565,male,2012-10-24 04:20:47+00:00,Bandung,21,-6.914212,107.600246
4998,36535293635,female,2012-10-13 08:20:08+00:00,Palembang,29,-2.985010,104.764139


In [26]:
pd.read_csv("events_with_users.csv")

,Unnamed: 0,event_id,start_time,city,lat,lng,yes_count,maybe_count,invited_count,no_count,total_users,weather_description,category,title,event_type,user_id
0,0,2587616435,2012-11-13 11:00:00.002000+00:00,Sihanoukville,10.633,103.500,93,65,317,47,522,Light Drizzle,Education & Learning,Sihanoukville Knowledge Festival 2025 - Mornin...,Indoor,47229975868
1,1,1145166049,2013-07-08 02:00:00+00:00,Palo Alto,37.442,-122.172,162,112,1021,150,1445,Cloudy,Business & Networking,Palo Alto Summer Night Networking Bash,Indoor,98905544751
2,2,920600431,2012-07-29 19:00:00+00:00,Karachi,24.893,67.028,22,8,993,26,1049,Cloudy,Entertainment,Karachi Nights: Summer Fiesta Under the Stars,Indoor,79515382063
3,3,3580637647,2012-10-22 10:00:00.003000+00:00,Los Angeles,3.156,101.612,63,42,430,20,555,Light Rain,Arts & Culture,Funky Art Walk LA - Morning Vibes & Visual Del...,Indoor,53769323787
4,4,1924180022,2012-11-11 20:00:00.003000+00:00,Palo Alto,37.416,-122.152,41,12,53,1,107,Cloudy,Arts & Culture,Palo Alto Art After Dark - A Fall Fiesta of Cr...,Indoor,58533063167
5,5,2396566946,2012-12-12 18:00:00.002000+00:00,Sihanoukville,10.633,103.500,391,275,1145,210,2021,Light Drizzle,Technology,Winter Tech Fest Sihanoukville 2025 - Innovate...,Indoor,50862885476
6,6,2654661184,2012-11-26 00:00:00.001000+00:00,Palo Alto,37.416,-122.152,7,6,16,1,30,Partly Cloudy,Seasonal & Festivals,Palo Alto Harvest Moon Lights Festival 2025,Outdoor,30054865988
7,7,229252687,2012-09-30 00:00:00.001000+00:00,Palo Alto,37.416,-122.152,155,102,767,246,1270,Partly Cloudy,Immersive Experiences,Palo Alto Nightscape - Immersive Experience Un...,Outdoor,10552930840
8,8,642644408,2012-10-31 00:00:00.001000+00:00,Siantar,2.965,99.075,283,111,3371,187,3952,Rain,Technology,"Siantar Tech Night Ignite 2025 - Innovate, Con...",Indoor,64462929029
9,9,4032767837,2012-11-30 16:00:00.003000+00:00,Jakarta,-6.262,106.837,31,29,283,21,364,Rain,Education & Learning,Empower Jakarta - Spring Learning Odyssey,Indoor,84057878038


In [ ]:
import pandas as pd
import numpy as np
import openai
import json
import time
from datetime import datetime, timedelta
import random
import os
import uuid

# Set your OpenAI API key
# openai.api_key = "your-api-key-here"  # Uncomment and add your key here
# Or set as environment variable: os.environ["OPENAI_API_KEY"] = "your-api-key"

def generate_user_batch(batch_size=50, total_users=5000, gender_distribution={'male': 0.55, 'female': 0.45}, mean_age=22):
    """Generate a batch of synthetic users using LLM"""
    
    # Create prompt for LLM
    prompt = f"""
    Generate {batch_size} realistic user profiles in JSON format.
    
    Each user should have:
    - A unique ID (UUID format)
    - Gender (approximately {gender_distribution['male']*100}% male, {gender_distribution['female']*100}% female)
    - Join date (between 1 and 3 years ago in YYYY-MM-DD format)
    - Location (city)
    - Age (normally distributed around mean={mean_age}, mostly between 18-40)
    - Latitude and longitude coordinates appropriate for their location
    
    Ensure the data is realistic and properly formatted.
    Return ONLY the JSON array, with no additional text.
    Example of the expected format:
    [
        {{
            "user_id": "a1b2c3d4-e5f6-4a5b-8c7d-9e0f1a2b3c4d",
            "gender": "female",
            "joinedAt": "2022-05-15",
            "location": "Austin, TX",
            "age": 23,
            "lat": 30.2672,
            "lng": -97.7431
        }},
        ...
    ]
    """
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Or use "gpt-3.5-turbo" for a lower-cost option
            messages=[{"role": "system", "content": "You are a data scientist creating synthetic user profiles."}, 
                      {"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=2000
        )
        
        user_data = json.loads(response.choices[0].message.content)
        return user_data
    except Exception as e:
        print(f"Error generating users: {e}")
        # Return empty list if there's an error
        return []

def generate_all_users(total_users=5000, batch_size=50):
    """Generate all synthetic users in batches"""
    all_users = []
    batches_needed = (total_users + batch_size - 1) // batch_size  # Ceiling division
    
    for i in range(batches_needed):
        print(f"Generating batch {i+1}/{batches_needed}...")
        
        # For the last batch, adjust batch size if necessary
        current_batch_size = min(batch_size, total_users - len(all_users))
        
        batch = generate_user_batch(current_batch_size)
        all_users.extend(batch)
        
        # Add a pause to avoid API rate limits
        time.sleep(1)
    
    return all_users[:total_users]  # Ensure we have exactly the requested number

def generate_coordinates_fallback(location):
    """Generate random coordinates as a fallback if LLM doesn't provide valid ones"""
    # Rough approximation of US coordinates
    lat = np.random.uniform(25, 49)
    lng = np.random.uniform(-125, -65)
    return lat, lng

def assign_users_to_events(events_df, users_df):
    """Assign users to events based on proximity and preferences"""
    events_with_users = events_df.copy()
    
    # Convert coordinates to numeric if they aren't already
    for df in [events_with_users, users_df]:
        for col in ['lat', 'lng']:
            if df[col].dtype == 'object':
                df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Make sure timestamps are datetime objects
    events_with_users['start_time'] = pd.to_datetime(events_with_users['start_time'])
    users_df['joinedAt'] = pd.to_datetime(users_df['joinedAt'])
    
    # List to store assigned user IDs
    assigned_users = []
    
    for _, event in events_with_users.iterrows():
        # Filter users who joined before the event
        eligible_users = users_df[users_df['joinedAt'] < event['start_time']]
        
        if len(eligible_users) == 0:
            # If no eligible users, use random user
            assigned_user = users_df.sample(1)['user_id'].iloc[0]
        else:
            # Calculate distance from event to each user
            eligible_users['distance'] = np.sqrt(
                (eligible_users['lat'] - event['lat'])**2 + 
                (eligible_users['lng'] - event['lng'])**2
            )
            
            # Higher probability for users closer to the event
            weights = 1 / (eligible_users['distance'] + 0.1)  # Add small constant to avoid division by zero
            
            # Sample user based on weights
            assigned_user = eligible_users.sample(1, weights=weights)['user_id'].iloc[0]
        
        assigned_users.append(assigned_user)
    
    # Add assigned users to events dataframe
    events_with_users['user_id'] = assigned_users
    
    return events_with_users

def clean_and_validate_users(users):
    """Clean and validate user data returned by the LLM"""
    cleaned_users = []
    
    for user in users:
        # Ensure all required fields are present
        required_fields = ['user_id', 'gender', 'joinedAt', 'location', 'age', 'lat', 'lng']
        if not all(field in user for field in required_fields):
            # Generate missing fields
            if 'user_id' not in user:
                user['user_id'] = str(uuid.uuid4())
            if 'gender' not in user:
                user['gender'] = random.choice(['male', 'female'])
            if 'joinedAt' not in user:
                days_ago = random.randint(30, 1095)  # 1-3 years
                user['joinedAt'] = (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
            if 'location' not in user:
                user['location'] = "Unknown"
            if 'age' not in user:
                user['age'] = int(np.random.normal(22, 5))
            if 'lat' not in user or 'lng' not in user:
                user['lat'], user['lng'] = generate_coordinates_fallback(user.get('location', ''))
        
        # Convert types
        try:
            user['age'] = int(user['age'])
            user['lat'] = float(user['lat'])
            user['lng'] = float(user['lng'])
        except (ValueError, TypeError):
            # Generate new values if conversion fails
            user['age'] = int(np.random.normal(22, 5))
            user['lat'], user['lng'] = generate_coordinates_fallback(user.get('location', ''))
        
        # Validate age range
        if user['age'] < 18 or user['age'] > 65:
            user['age'] = int(np.clip(user['age'], 18, 65))
        
        cleaned_users.append(user)
    
    return cleaned_users

def generate_fallback_users(n=5000):
    """Generate users without using LLM as a fallback"""
    users = []
    now = datetime.now()
    
    for _ in range(n):
        gender = random.choice(['male', 'female'])
        days_ago = random.randint(30, 1095)  # 1-3 years
        join_date = (now - timedelta(days=days_ago)).strftime('%Y-%m-%d')
        age = int(np.random.normal(22, 5))
        age = max(18, min(65, age))  # Clip between 18 and 65
        lat = np.random.uniform(25, 49)
        lng = np.random.uniform(-125, -65)
        
        users.append({
            'user_id': str(uuid.uuid4()),
            'gender': gender,
            'joinedAt': join_date,
            'location': 'Unknown Location',
            'age': age,
            'lat': lat,
            'lng': lng
        })
    
    return users

def main():
    # Load events data
    try:
        events_df = pd.read_csv('events_data.csv')
        print(f"Loaded {len(events_df)} events")
    except FileNotFoundError:
        print("Events data file not found. Please provide the correct path.")
        return
    
    try:
        print("Generating synthetic users using LLM...")
        users = generate_all_users(total_users=5000, batch_size=50)
        users = clean_and_validate_users(users)
        
        # If we didn't get enough users from the LLM, generate the rest
        if len(users) < 5000:
            print(f"Only generated {len(users)} users with LLM. Generating {5000 - len(users)} more without LLM...")
            fallback_users = generate_fallback_users(5000 - len(users))
            users.extend(fallback_users)
        
    except Exception as e:
        print(f"Error using LLM to generate users: {e}")
        print("Falling back to non-LLM generation...")
        users = generate_fallback_users(5000)
    
    # Convert to DataFrame
    users_df = pd.DataFrame(users)
    
    # Ensure gender distribution is approximately correct
    gender_counts = users_df['gender'].value_counts()
    print(f"Gender distribution: {gender_counts}")
    
    # Assign users to events
    print("Assigning users to events...")
    events_with_users = assign_users_to_events(events_df, users_df)
    
    # Save results
    users_df.to_csv('synthetic_users.csv', index=False)
    events_with_users.to_csv('events_with_users.csv', index=False)
    
    print(f"Generated {len(users_df)} users")
    print(f"Assigned users to {len(events_with_users)} events")
    print("Files saved: synthetic_users.csv, events_with_users.csv")
    
    return users_df, events_with_users

if __name__ == "__main__":
    main()

In [ ]:
!pip install faker

In [66]:
print(users_df["gender"].value_counts())

gender
male      5203
female    4321
Name: count, dtype: int64


In [22]:
users_2_gen_df = users_df[~users_df['user_id'].isin(unique_users)]
users_gen_df = users_df[users_df['user_id'].isin(unique_users)]
users_2_gen_df.head(2)

,Unnamed: 0,user_id,gender,joinedAt,location,age,lat,lng
1,1,3537982273,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20,3.59,98.674
2,2,3473687777,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47,3.59,98.674


In [63]:
len(users_2_gen_df)

4209

In [76]:


# Ensure users_2_gen_df is a copy of the original DataFrame to avoid SettingWithCopyWarning
users_2_gen_df = users_df[~users_df['user_id'].isin(unique_users)].copy()

# Use .loc to safely assign values
users_2_gen_df.loc[:, "user_id2"] = ids_2_gen  # Assign ids_2_gen to a new column user_id2

# Drop the original user_id column
users_2_gen_df = users_2_gen_df.drop("user_id", axis=1)

# Rename the user_id2 column to user_id
users_2_gen_df.rename(columns={"user_id2": "user_id"}, inplace=True)

# Display the first two rows to verify changes
users_2_gen_df.head(2)

,Unnamed: 0,gender,joinedAt,location,age,lat,lng,user_id
1,1,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20,3.59,98.674,3320380166
2,2,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47,3.59,98.674,3810793697


In [ ]:
interactions_df
# Ensure users_2_gen_df is a copy of the original DataFrame to avoid SettingWithCopyWarning
users_2_gen_df = users_df[~users_df['user_id'].isin(unique_users)].copy()

# Use .loc to safely assign values
users_2_gen_df.loc[:, "user_id2"] = ids_2_gen  # Assign ids_2_gen to a new column user_id2

# Drop the original user_id column
users_2_gen_df = users_2_gen_df.drop("user_id", axis=1)

# Rename the user_id2 column to user_id
users_2_gen_df.rename(columns={"user_id2": "user_id"}, inplace=True)

# Display the first two rows to verify changes
users_2_gen_df.head(2)

In [77]:
len(users_2_gen_df)


4209

In [86]:
users_2_gen_df =users_2_gen[["event_id","user_id"]]\
    .merge(users_2_gen_df,on="user_id",how="inner")
    


users_2_gen_df.head(2)

,event_id,user_id,Unnamed: 0,gender,joinedAt,location,age,lat,lng
0,1186208412,3320380166,1,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20,3.59,98.674
1,1186208412,3810793697,2,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47,3.59,98.674


In [80]:
main_user_data = pd.concat([users_gen_df,users_2_gen_df],axis=0)
main_user_data = main_user_data.drop("Unnamed: 0",axis=1)
main_user_data.head()

,user_id,gender,joinedAt,location,age,lat,lng
0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.590,98.674
4,1091720544,female,2012-10-30T11:55:19.469Z,Plaine Des Papayes Pamplemousses Mauritius,24,-20.063,57.581
5,3341263967,female,2012-09-21T14:33:02.704Z,Toronto Ontario,24,43.653,-79.384
6,1363661070,female,2012-10-31T02:21:02.061Z,Phnom Penh,18,11.573,104.858
7,3104149470,male,2012-10-18T15:50:49.503Z,Medan Indonesia,20,3.590,98.674


In [ ]:
users_2_gen_df =users_2_gen[["event_id","user_id"]]\
    .merge(users_2_gen_df,on="user_id",how="inner")
    
users_2_gen_df.head(2)

In [34]:

events_df[events_df["event_id"]==1186208412]

,event_id,city,category
11223,1186208412,Toronto,Education & Learning


In [100]:
len(users_2_gen), len(users_2_gen_df)

(6955602, 128276)

In [ ]:
interactions_df = 

In [98]:
users_2_gen_df.merge(interactions_df,on=["event_id","user_id"],how="inner")
users_2_gen_df.head(2)

,event_id,user_id,Unnamed: 0,gender,joinedAt,location,age,lat,lng
0,1186208412,3320380166,1,male,2012-09-29T18:03:12.111Z,Medan Indonesia,20,3.59,98.674
1,1186208412,3810793697,2,female,2012-10-03T12:19:29.975Z,Medan Indonesia,47,3.59,98.674


In [107]:
col2 = ["event_id", "category"]
events_df = events_df[col2]

# Corrected merge to include all three DataFrames
interaction_users_events = interactions_df[['event_id','user_id']].merge(users_df, on='user_id', how='inner') \
                                           .merge(events_df, on='event_id', how='inner')

len(interaction_users_events)

28249

In [108]:
interaction_users_events['event_id'].nunique(), interaction_users_events['user_id'].nunique()

(4967, 5315)

In [109]:
new_interactions_data = pd.concat([interaction_users_events,])

In [110]:
interaction_users_events

,event_id,user_id,gender,category
0,702719295,3468617687,female,Food & Drink
1,702719295,487910947,male,Food & Drink
2,702719295,2769572227,female,Food & Drink
3,702719295,2974942354,male,Food & Drink
4,3093414446,4137863751,female,Technology
...,...,...,...,...
28244,439904567,853433602,male,Technology
28245,439904567,810729686,female,Technology
28246,439904567,3830152023,female,Technology
28247,3075319534,2266389049,male,Seasonal & Festivals


In [60]:
interactions.category.nunique()

12

In [8]:
import pandas as pd
import json

# Define the event categories
EVENT_CATEGORIES = [
    'Music & Concerts',      # Covers live music, DJ performances, outdoor concerts
    'Food & Drink',          # Covers wine/beer tastings, food festivals, mixology, food truck rallies
    'Education & Learning',  # Covers workshops, educational events, book launches
    'Sports & Fitness',      # Covers sports viewing, fitness challenges
    'Arts & Culture',        # Covers art exhibits, photography, fashion, poetry, cultural celebrations
    'Business & Networking', # Covers networking, industry roundtables, product launches
    'Technology',            # Covers tech conferences, gaming tournaments
    'Community & Causes',    # Covers charity fundraisers, community gatherings, farmers markets
    'Health & Wellness',     # Covers wellness retreats, cooking workshops (health-focused)
    'Entertainment',         # Covers comedy, trivia, karaoke, film screenings
    'Seasonal & Festivals',  # Covers seasonal celebrations, street festivals, craft fairs
    'Immersive Experiences'  # Covers immersive events that don't fit other categories
]

def get_user_interests(interactions_df):
    """
    Get unique event categories for each user and determine their interests.
    
    Parameters:
    - interactions_df: DataFrame containing user-event interactions with category information
    
    Returns:
    - DataFrame with original data plus user_interests column
    """
    # Create a copy of the dataframe to avoid modifying the original
    df = interactions.copy()
    
    # Step 1: Group by user_id and collect unique categories
    # Convert to strings first to avoid errors with lists
    user_categories_dict = {}
    
    # Group by user_id and get unique categories
    for user_id, group in df.groupby('user_id'):
        # Get unique categories as a list
        interests = group['category'].unique().tolist()
        # Filter to only include categories from EVENT_CATEGORIES
        #interests = [cat for cat in categories if cat in EVENT_CATEGORIES]
        # Store as a string representation for now
        user_categories_dict[user_id] = json.dumps(interests)
    
    # Step 2: Add user_interests to the original dataframe
    # Create a function to map user_id to interests
    def get_interests(user_id):
        interests_json = user_categories_dict.get(user_id, '[]')
        return json.loads(interests_json)
    
    # Apply the function to create the user_interest column
    df['user_interest_str'] = df['user_id'].apply(lambda x: user_categories_dict.get(x, '[]'))
    df['user_interest'] = df['user_interest_str'].apply(json.loads)
    
    # Remove the intermediate string column if you don't need it
    df = df.drop(columns=['user_interest_str'])
    
    return df

def analyze_user_interests(df):
    """
    Analyze and print statistics about user interests.
    
    Parameters:
    - df: DataFrame with user_interest column
    """
    # Count the number of unique users
    unique_users = df['user_id'].nunique()
    
    # Create a temporary dataframe with user_id and a stringified version of interests
    temp_df = df[['user_id', 'user_interest']].drop_duplicates('user_id')
    
    # Count users with interests
    users_with_interests = sum(temp_df['user_interest'].apply(lambda x: len(x) > 0))
    
    # Calculate most common interests by flattening all interest lists
    all_interests = []
    for interests in temp_df['user_interest']:
        all_interests.extend(interests)
    
    # Count occurrences of each interest
    interest_counts = pd.Series(all_interests).value_counts()
    
    print(f"Total unique users: {unique_users}")
    print(f"Users with identified interests: {users_with_interests}")
    print("\nMost common user interests:")
    print(interest_counts)
    
    return interest_counts

def create_user_interests_summary(df):
    """
    Create a summarized dataframe of user interests without list objects
    
    Parameters:
    - df: DataFrame with user_interest column (containing lists)
    
    Returns:
    - DataFrame with user_id and interest counts
    """
    # Get unique users
    user_df = df[['user_id', 'user_interest']].drop_duplicates('user_id')
    
    # Create summary statistics
    summary_data = []
    
    for _, row in user_df.iterrows():
        user_id = row['user_id']
        interests = row['user_interest']
        
        # Create a row for the summary
        user_summary = {
            'user_id': user_id,
            'num_interests': len(interests),
            'interests_joined': ', '.join(interests)
        }
        
        # Add individual interest flags
        for category in EVENT_CATEGORIES:
            user_summary[f'likes_{category.lower().replace(" & ", "_").replace(" ", "_")}'] = int(category in interests)
        
        summary_data.append(user_summary)
    
    return pd.DataFrame(summary_data)

if __name__ == "__main__":
    
    # Process the sample data
    enriched_df = get_user_interests(interactions)
    
    # # Display the results - avoiding unique() operations on list columns
    # print("\nEnriched DataFrame with User Interests:")
    # for user_id in enriched_df['user_id'].unique():
    #     user_data = enriched_df[enriched_df['user_id'] == user_id].iloc[0]
    #     print(f"User {user_id}: {user_data['user_interest']}")
    
    # # Create a summarized view for analysis that avoids list objects
    # user_summary = create_user_interests_summary(enriched_df)
    # print("\nUser Interests Summary (hashable format):")
    # print(user_summary)
    
    # # Analyze user interests using our safe method
    # print("\nUser Interest Analysis:")
    # interest_stats = analyze_user_interests(enriched_df)

In [9]:
enriched_df.head()

,Unnamed: 0,event_id,user_id,interaction_type,interaction_label,category,user_interest
0,180,1186208412,3320380166,maybe,1,Education & Learning,"[Education & Learning, Business & Networking]"
1,181,1186208412,3810793697,maybe,1,Education & Learning,"[Education & Learning, Seasonal & Festivals, B..."
2,182,1186208412,1379121209,invited & maybe,1,Education & Learning,[Education & Learning]
3,183,1186208412,440668682,invited & maybe,1,Education & Learning,[Education & Learning]
4,184,1186208412,1728988561,no,0,Education & Learning,"[Education & Learning, Entertainment, Business..."


In [10]:
len(users_df)

9524

In [11]:
users_data = users_df.merge(enriched_df[['user_interest','user_id']], on='user_id',how='inner')
print(len(users_data))
users_data.head()

28249


,Unnamed: 0,user_id,gender,joinedAt,location,age,lat,lng,user_interest
0,0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.59,98.674,"[Technology, Seasonal & Festivals, Health & We..."
1,0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.59,98.674,"[Technology, Seasonal & Festivals, Health & We..."
2,0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.59,98.674,"[Technology, Seasonal & Festivals, Health & We..."
3,0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.59,98.674,"[Technology, Seasonal & Festivals, Health & We..."
4,0,3197468391,male,2012-10-02T06:40:55.524Z,Medan Indonesia,19,3.59,98.674,"[Technology, Seasonal & Festivals, Health & We..."


In [12]:
users_data['user_id'].nunique()

5315